<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
      Fraud, Waste, and Abuse (FWA) Detection - Dataprocessing<br>
       <img id="teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>

<p style = 'font-size:20px;font-family:Arial'><b>Introduction</b><br>
<p style = 'font-size:16px;font-family:Arial'>In this notebook we will process the source data and create the tables required for reporting. </p>

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial'>1. Connect to Vantage</b>
<p style = 'font-size:16px;font-family:Arial'>You will be prompted to provide the password. Enter your password, press the Enter key, and then use the down arrow to go to the next cell.</p>

In [ ]:
%connect local, hidewarnings=True

<p style = 'font-size:16px;font-family:Arial'>Setup for execution of notebook. Begin running steps with Shift + Enter keys.</p>

In [ ]:
Set query_band='DEMO=FF_FWA_Healthcare_Sql.ipynb;' update for session;

<p style = 'font-size:18px;font-family:Arial'><b>Getting Data for This Demo</b></p>
<p style = 'font-size:16px;font-family:Arial'>We have provided data for this demo on cloud storage. You can either run the demo using foreign tables to access the data without any storage on your environment or download the data to local storage, which may yield faster execution. Still, there could be considerations of available storage. Two statements are in the following cell, and one of them is commented out. You may switch between the modes by changing the comment string.</p>

In [ ]:
call get_data('DEMO_HealthcareFWA_local');           -- Takes approx 7minutes
-- call get_data('DEMO_HealthcareFWA_cloud');           -- Takes 10 seconds

<p style = 'font-size:16px;font-family:Arial'>Optional step – if you want to see status of databases/tables created and space used.</p>

In [ ]:
call space_report();          -- Takes 5 seconds

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial'>2. Reporting tables for Musculoskeletal Conditions</b>
<p style = 'font-size:16px;font-family:Arial'>Here we are creating reporting tables needed for the musculoskeletal condtion reporting. As the first step we are creating and intermediate view which will hold the patient ids and will work as a driver table in the subsequent steps.In this view we have converted all the procedure codes to its corresponding procedure descriptions.</p>

In [ ]:
REPLACE VIEW DEMO_USER.WC_BK_V_NPATH_QRY AS (
SELECT
        patient_identifier,
        HCPCS_Line_Procedure_Billed_Code,
        Total_Amount_Paid_Per_Line,
        Service_Line_From_Date_ Service_Line_From_Date,
        first_musc_diag,
        Employee_Mailing_City,
        CASE 
          WHEN REGEXP_SIMILAR(HCPCS_Line_Procedure_Billed_Code,'9894[0-4]','c') = 1 
              THEN 'CMT'
          WHEN REGEXP_SIMILAR(HCPCS_Line_Procedure_Billed_Code,'9716[1-4]','c') = 1
              THEN 'PT'
          WHEN REGEXP_SIMILAR(HCPCS_Line_Procedure_Billed_Code,'9716[5-8]','c') = 1
              THEN 'OT'
          WHEN REGEXP_SIMILAR(HCPCS_Line_Procedure_Billed_Code,'970((1[0-9])|(2[0-8]))','c') = 1
              THEN 'SM'
          WHEN HCPCS_Line_Procedure_Billed_Code in ('63047', '63048', '63030', '63035',
                                                    '22612', '22633', '22558', '22551',
                                                    '64635', '64636', '63045') THEN 'SRG'
          WHEN HCPCS_Line_Procedure_Billed_Code in ('70551','70553','70544','70547',
                                                    '70549','70540','70543','70540',
                                                    '70543','70336') THEN 'BCK_NCK_MRI'
          WHEN HCPCS_Line_Procedure_Billed_Code in ('72141','72156','72146','72157',
                                                     '72148','72158') THEN 'SPINE_MRI'
          WHEN HCPCS_Line_Procedure_Billed_Code in ('77059','77021') THEN 'BREAST_MRI'
          WHEN HCPCS_Line_Procedure_Billed_Code in ('73221','73223','73721','73723') THEN 'JOINTS_MRI'
          WHEN HCPCS_Line_Procedure_Billed_Code in ('73718','73720','73218','73220') THEN 'EXTR_MRI'
          WHEN HCPCS_Line_Procedure_Billed_Code in ('74181','74183','74185') THEN 'ABD_MRI'
          WHEN HCPCS_Line_Procedure_Billed_Code in ('72195','72197','72195',
                                                    '72197','72195','72197') THEN 'PLVS_MRI'
          WHEN HCPCS_Line_Procedure_Billed_Code in ('72125','72126','72131',
                                                    '72132''72128','72129') THEN 'CT_SCAN'
          WHEN HCPCS_Line_Procedure_Billed_Code in ('72040','72050','72220',
                                                    '72100','72110','72114',
                                                    '72200','72202','72052',
                                                    '72082','72070') THEN 'XRAY'
          ELSE 'OTHER' END as Description
          ,Billing_Provider_Last_Name_Or_Group,Billing_Provider_Fein
          
    FROM(

SELECT
          (Claim_Administrator_Claim_Number || '_' ||Insurer_FEIN) as patient_identifier,
          HCPCS_Line_Procedure_Billed_Code, Total_Amount_Paid_Per_Line, 
          CAST(Service_Line_From_Date AS DATE FORMAT 'YYYYMMDD') Service_Line_From_Date_,
          Employee_Mailing_City,
          min(Service_Line_From_Date_) OVER (PARTITION BY patient_identifier) first_musc_diag
          ,Billing_Provider_Last_Name_Or_Group,Billing_Provider_Fein
      FROM DEMO_HealthcareFWA.WC_SV1_MSK
UNION ALL
SELECT 
          (Claim_Administrator_Claim_Number || Insurer_FEIN) as patient_identifier,
          HCPCS_Line_Procedure_Billed_Code, CAST(Total_Amount_Paid_Per_Line as FLOAT), 
          CAST(Service_Line_From_Date AS DATE FORMAT 'YYYYMMDD') Service_Line_From_Date_,
          Employee_Mailing_City,
          min(Service_Line_From_Date_) OVER (PARTITION BY patient_identifier) first_musc_diag
          ,Billing_Provider_Last_Name_Or_Group,Billing_Provider_Fein
      FROM DEMO_HealthcareFWA.WC_SV2_MSK
)
 d
    WHERE Description not like 'OTHER'
)
;

In [ ]:
select top 10 * from WC_BK_V_NPATH_QRY;

<p style = 'font-size:16px;font-family:Arial'>Here we are calculating the data for the surgical pathways i.e where the last procedure is Surgery (SRG) augmenting it with the cost of each individual procedure and the overall cost. We will use Vantage's NPath function for this. As we have data available for the TX county only we have added that in filter clause. </p>

In [ ]:
CREATE TABLE DEMO_USER.surg_pathways
AS (
SELECT
    TRIM (BOTH FROM REGEXP_REPLACE(p.patient_identifier, '[\[\]]', '', 1, 0, 'c')) as trimmed_claim,
    REGEXP_REPLACE(path, '[\[\]\s]+', '', 1, 0, 'c') AS path,
    first_musc_diag,
    last_date,
    REGEXP_REPLACE(dates, '[\[\]\s]+', '', 1, 0, 'c') AS dates,
    REGEXP_REPLACE(cost, '[\[\]\s]+', '', 1, 0, 'c') AS cost,
    length,
    REGEXP_REPLACE(prov_group, '[\[\]]+', '', 1, 0, 'c') as prov_group,
    REGEXP_REPLACE(provider, '[\[\]\s]+', '', 1, 0, 'c') as provider,
    county,
    total_cost
FROM nPath (
  ON (
      SELECT
          patient_identifier, Description, Service_Line_From_Date, first_musc_diag,
          CAST(AVG(Total_amount_paid_Per_Line) as decimal(10,2)) as Total_Charge_Per_Line
          ,Billing_Provider_Last_Name_or_Group, Billing_Provider_FEIN
      FROM DEMO_user.WC_BK_V_NPATH_QRY
      GROUP BY patient_identifier, Description, first_musc_diag, Service_Line_From_Date,
               Billing_Provider_Last_Name_or_Group, Billing_Provider_FEIN)
  PARTITION BY patient_identifier
  ORDER BY Service_Line_From_Date
  USING
  Mode (nonoverlapping)
  Pattern ('^N_SRG+.SRG') 
  Symbols (
      Description not like 'SRG' as N_SRG,
      Description like 'SRG' as SRG
  )
  Result (

      FIRST (patient_identifier OF ANY( SRG, N_SRG)) AS patient_identifier,
      ACCUMULATE (CAST (Description AS CHAR(40)) OF ANY( SRG, N_SRG)) AS path,
      LAST(Service_Line_From_Date of any(SRG, N_SRG)) as last_date,
      FIRST(first_musc_diag of any(SRG, N_SRG)) as first_musc_diag,
      ACCUMULATE (Service_Line_From_Date of any(SRG, N_SRG)) AS dates,
      ACCUMULATE(Total_Charge_Per_Line of ANY(  SRG, N_SRG)) as cost,
      COUNT(* of ANY( SRG, N_SRG)) as length,
      ACCUMULATE(Billing_Provider_Last_Name_or_Group of ANY(SRG, N_SRG)) as prov_group,
      ACCUMULATE(Billing_Provider_FEIN of ANY(SRG, N_SRG)) as provider
  )
) p
LEFT JOIN (
    select 
        MIN(COUNTY) as county,
        MIN(STATE_CODE) as state_code,
        SUM(Total_Charge_Per_Line) as total_cost,
        (Claim_Administrator_Claim_Number ||'_'|| Insurer_FEIN) as patient_identifier
    from DEMO_HealthcareFWA.WC_SV1_MSK
    inner join DEMO_HealthcareFWA.wc_us_cities
    on lower(Employee_Mailing_City) = lower(CITY)
    where Employee_Mailing_City is not null and STATE_CODE = 'TX'
    group by patient_identifier
) d
ON trimmed_claim = d.patient_identifier
)WITH DATA ;

<p style = 'font-size:16px;font-family:Arial'>Next we are calculating the data for all musculoskeletal procedures whcih are not ending with a surgery. Similar to above we are augmenting it with the cost of each individual procedure and the overall cost.</p>

In [ ]:
CREATE TABLE DEMO_USER.nsurg_pathways  as (
SELECT 
    a.*,
    COUNTY,
    STATE_CODE
FROM
(
    SELECT 
        TRIM (BOTH FROM REGEXP_REPLACE(p.patient_identifier, '[\[\]]', '', 1, 0, 'c')) as trimmed_claim,
        REGEXP_REPLACE(path, '[\[\]\s]+', '', 1, 0, 'c')||',NSRG' AS path,
        first_musc_diag,    
        last_date,
        REGEXP_REPLACE(dates, '[\[\]\s]+', '', 1, 0, 'c')||','||last_date AS dates,
        REGEXP_REPLACE(cost, '[\[\]\s]+', '', 1, 0, 'c')||',0.0' AS cost,    
        length
        FROM nPath (
    ON (
      SELECT
        patient_identifier, Description, Service_Line_From_Date, first_musc_diag,
        CAST(AVG(Total_amount_paid_Per_Line) as decimal(10,2)) as Total_Charge_Per_Line
      FROM DEMO_USER.WC_BK_V_NPATH_QRY
      GROUP BY patient_identifier, Description, first_musc_diag, Service_Line_From_Date)
    PARTITION BY patient_identifier
    ORDER BY Service_Line_From_Date
    USING
    Mode (nonoverlapping)
    Pattern ('^N_SRG+.N_SRG$') 
    Symbols (
        Description not like 'SRG' as N_SRG
    )
    Result (
        FIRST (patient_identifier OF ANY( N_SRG)) AS patient_identifier,
        ACCUMULATE (CAST (Description AS CHAR(40)) OF ANY( N_SRG)) AS path,
        LAST(Service_Line_From_Date of any(N_SRG)) as last_date,
        FIRST(first_musc_diag of any(N_SRG)) as first_musc_diag,      
        ACCUMULATE (Service_Line_From_Date of any(N_SRG)) AS dates,
        ACCUMULATE(Total_Charge_Per_Line of ANY(N_SRG)) as cost,
        COUNT(* of ANY( N_SRG)) as length
    )
) p
) a
LEFT JOIN (
    select MIN(COUNTY) as COUNTY, MIN(STATE_CODE) as STATE_CODE, 
        (Claim_Administrator_Claim_Number ||'_'|| Insurer_FEIN) as patient_identifier
    from DEMO_HealthcareFWA.WC_SV1HEADER
    inner join DEMO_HealthcareFWA.wc_us_cities
    on lower(Employee_Mailing_City) = lower(CITY)
    where Employee_Mailing_City is not null and STATE_CODE = 'TX'
    group by patient_identifier
) d
ON trimmed_claim = d.patient_identifier
) WITH DATA ;             

<p style = 'font-size:16px;font-family:Arial'>In the table below, we are getting all the providers for the MSK procedures using the billing information.</p>

In [ ]:
CREATE TABLE DEMO_USER.msk_pathwys_prov  as (
SELECT
    Billing_Provider_FEIN,
    SUM(CASE WHEN HCPCS_Line_Procedure_Billed_Code in ('70551','70553','70544','70547','70549',
                               '70540','70543','70540','70543','70336',
                               '72141','72156','72146','72157','72148',
                               '72158','77059','77021','73221','73223',
                               '73721','73723','73718','73720','73218',
                               '73220','74181','74183','74185','72195',
                               '72197','72195','72197','72195','72197') 
        THEN 1 ELSE 0 END) as "MRI Count",
    CAST(SUM(Total_Charge_Per_Line) AS DECIMAL(20,2)) as "Total Cost"
FROM (
  select 
      CAST(Total_Charge_Per_Line AS FLOAT) as Total_Charge_Per_Line, Billing_Provider_FEIN,
      HCPCS_Line_Procedure_Billed_Code
  from DEMO_HealthcareFWA.WC_SV1_MSK
UNION ALL
  select
      CAST(Total_Charge_Per_Line AS FLOAT) as Total_Charge_Per_Line, Billing_Provider_FEIN,
      HCPCS_Line_Procedure_Billed_Code
  from DEMO_HealthcareFWA.WC_SV2_MSK
) d
GROUP BY Billing_Provider_FEIN
) WITH DATA ;          

<p style = 'font-size:16px;font-family:Arial'>In the table below, we are claculating the average surgery costs.</p>

In [ ]:
CREATE TABLE DEMO_USER.srg_averages  as (
SELECT
    avg(first_surg - first_musc_diag) as avg_days_srg, 
    avg(cost) as avg_cost, 
    count(patient_identifier) as volume
FROM (
    SELECT
        MIN(CASE WHEN 
            HCPCS_Line_Procedure_Billed_Code in ('63047','63048','63030','63035','22612','22633',
                                                 '22558','22551','64635','64636','63045')
            THEN Service_Line_From_Date END) as first_surg,
        MIN(CASE WHEN 
            HCPCS_Line_Procedure_Billed_Code in ('70551','70553','70544','70547','70549','70540','70543','70540',
                                                 '70543','70336','72141','72156','72146','72157','72148','72158',
                                                 '77059','77021','73221','73223','73721','73723','73718','73720',
                                                 '73218','73220','74181','74183','74185','72195','72197','72195',
                                                 '72197','72195','72197')
            THEN Service_Line_From_Date END) as first_mri,
        MIN(Service_Line_From_Date) as first_musc_diag,
        MIN(CASE WHEN 
            HCPCS_Line_Procedure_Billed_Code in ('70551','70553','70544','70547','70549','70540','70543','70540',
                                                 '70543','70336','72141','72156','72146','72157','72148','72158',
                                                 '77059','77021','73221','73223','73721','73723','73718','73720',
                                                 '73218','73220','74181','74183','74185','72195','72197','72195',
                                                 '72197','72195','72197')
            THEN 1 ELSE 0 END) as mri_count,
        CASE 
            WHEN (first_mri <= first_musc_diag + (7*6)) THEN 'Non EBM'
            ELSE 'EBM'
        END as label,
        SUM(Total_Charge_Per_Line) as cost,
        patient_identifier
    FROM
    (
      select
          CAST(Total_Charge_Per_Line AS FLOAT) as Total_Charge_Per_Line,
          (Claim_Administrator_Claim_Number || '_' || Insurer_FEIN) as patient_identifier,
          HCPCS_Line_Procedure_Billed_Code, Service_Line_From_Date
      from DEMO_HealthcareFWA.WC_SV1_MSK

      UNION ALL
      select
          CAST(Total_Charge_Per_Line AS FLOAT) as Total_Charge_Per_Line,
          (Claim_Administrator_Claim_Number || '_' || Insurer_FEIN) as patient_identifier,
          HCPCS_Line_Procedure_Billed_Code, CAST(Service_Line_From_Date as DATE FORMAT 'YYYYMMDD')
      from DEMO_HealthcareFWA.WC_SV2_MSK
    ) d
    GROUP BY patient_identifier
) d
WHERE first_surg is not null
) WITH DATA ;                   

<p style = 'font-size:16px;font-family:Arial'>In the table below, we are calculating average mri's performed on patients.</p>

In [ ]:
CREATE TABLE DEMO_USER.srg_mri  as (
SELECT label, AVG(mri_count) avg_mri_count
FROM (
    SELECT
        MIN(CASE WHEN 
            HCPCS_Line_Procedure_Billed_Code in ('63047','63048','63030','63035','22612','22633',
                                                 '22558','22551','64635','64636','63045')
            THEN Service_Line_From_Date END) as first_surg,
        MIN(CASE WHEN 
            HCPCS_Line_Procedure_Billed_Code in ('70551','70553','70544','70547','70549','70540','70543','70540',
                                                 '70543','70336','72141','72156','72146','72157','72148','72158',
                                                 '77059','77021','73221','73223','73721','73723','73718','73720',
                                                 '73218','73220','74181','74183','74185','72195','72197','72195',
                                                 '72197','72195','72197')
            THEN Service_Line_From_Date END) as first_mri,
        MIN(Service_Line_From_Date) as first_musc_diag,
        SUM(CASE WHEN 
            HCPCS_Line_Procedure_Billed_Code in ('70551','70553','70544','70547','70549','70540','70543','70540',
                                                 '70543','70336','72141','72156','72146','72157','72148','72158',
                                                 '77059','77021','73221','73223','73721','73723','73718','73720',
                                                 '73218','73220','74181','74183','74185','72195','72197','72195',
                                                 '72197','72195','72197')
            THEN 1 ELSE 0 END) as mri_count,
        CASE 
            WHEN (first_mri <= first_musc_diag + (7*6)) THEN 'Non EBM'
            ELSE 'EBM'
        END as label,
        SUM(Total_Charge_Per_Line) as cost,
        patient_identifier
    FROM
    (
      select
          CAST(Total_Charge_Per_Line AS FLOAT) as Total_Charge_Per_Line,
          (Claim_Administrator_Claim_Number || '_' || Insurer_FEIN) as patient_identifier,
          HCPCS_Line_Procedure_Billed_Code, Service_Line_From_Date
      from DEMO_HealthcareFWA.WC_SV1_MSK

      UNION ALL
      select
          CAST(Total_Charge_Per_Line AS FLOAT) as Total_Charge_Per_Line,
          (Claim_Administrator_Claim_Number || '_' || Insurer_FEIN) as patient_identifier,
          HCPCS_Line_Procedure_Billed_Code, CAST(Service_Line_From_Date as DATE FORMAT 'YYYYMMDD')
      from DEMO_HealthcareFWA.WC_SV2_MSK
    ) d
    GROUP BY patient_identifier
) d
WHERE first_surg is not null
GROUP BY label
) WITH DATA ;                   

<p style = 'font-size:16px;font-family:Arial'>In the table below, we are getting cost of all the procedures in the musculoskeletal condition.</p>

In [ ]:
CREATE TABLE DEMO_USER.msk_comorb_conds  as (
SELECT *
FROM(
  select
  COALESCE(DESCRIPTION,CAST(Comorb_Code AS CHAR(3))) condition,
  count(Bill_ID) volume,
  sum(Total_Charge_Per_Line) cost
  from (
    sel Bill_ID, Comorb_Code, Total_Charge_Per_Line from
    (
        SELECT
            main.Bill_ID,
            main.First_ICD9_CM_Diagnosis_Code, main.Second_ICD9_CM_Diagnosis_Code,
            main.Third_ICD9_CM_Diagnosis_Code, main.Fourth_ICD9_CM_Diagnosis_Code, 
            main.Fifth_ICD9_CM_Diagnosis_Code, Total_Charge_Per_Line
        from DEMO_HealthcareFWA.WC_SV1HEADER main
        INNER JOIN DEMO_HealthcareFWA.WC_SV1DETAIL det
        ON main.Bill_ID = det.Bill_ID
        WHERE ((main.First_ICD9_CM_Diagnosis_Code like 'M__%') OR (main.Second_ICD9_CM_Diagnosis_Code like 'M__%')
                OR (main.Third_ICD9_CM_Diagnosis_Code like 'M__%') OR (main.Fourth_ICD9_CM_Diagnosis_Code like 'M__%')
                OR (main.Fifth_ICD9_CM_Diagnosis_Code like 'M__%'))
         ) a
    UNPIVOT EXCLUDE NULLS
    (
      Comorb_Code  FOR  Diag_level IN
        (First_ICD9_CM_Diagnosis_Code, Second_ICD9_CM_Diagnosis_Code,
        Third_ICD9_CM_Diagnosis_Code, Fourth_ICD9_CM_Diagnosis_Code,
        Fifth_ICD9_CM_Diagnosis_Code)
    ) Tmp
    where 
      (Comorb_Code not like 'M__%') AND (Comorb_Code not like 'S__%') AND
      (Comorb_Code not like 'R26%') AND (Comorb_Code not like 'G56%') AND
      (Comorb_Code not like 'Z47%') AND (Comorb_Code not like 'T84%') AND
      (Comorb_Code not like 'G82%') AND (Comorb_Code not like 'Y99%') AND
      (Comorb_Code not like 'T85%') AND (Comorb_Code not like 'Z96%') AND
      (Comorb_Code not like 'R60%') AND (Comorb_Code not like 'R29%') AND
      (Comorb_Code not like 'T14%') AND (Comorb_Code not like 'G57%') AND
      (Comorb_Code not like 'W01%') AND (Comorb_Code not like 'Y92%') AND
      (Comorb_Code not like 'W19%') AND (Comorb_Code not like 'X50%') AND
      (Comorb_Code not like 'G89%') AND (Comorb_Code not like 'Z79%') AND
      (Comorb_Code not like 'Z98%') AND (Comorb_Code not like 'Z48%') AND
      (Comorb_Code not like 'Z01%') AND (Comorb_Code not like 'Z51%') AND
      (Comorb_Code not like 'Z68%')
) A
LEFT JOIN DEMO_HealthcareFWA.WC_MSK_DIAG_DESC
ON CAST(Comorb_Code AS CHAR(3)) = ICD_10_CODE
group by Condition
) d
QUALIFY RANK() OVER (ORDER BY cost DESC) <= 10 OR
      RANK() OVER (ORDER BY volume DESC) <= 10
    ) WITH DATA ; 

<p style = 'font-size:16px;font-family:Arial'>In the table below, we are getting cost per percedure group for all the procedures in the musculoskeletal condition.</p>

In [ ]:
CREATE TABLE DEMO_USER.msk_proc_groups  as (
SELECT "group", cost
FROM(
    SELECT
        SUM(CASE WHEN REGEXP_SIMILAR(HCPCS_Line_Procedure_Billed_Code,'9894[0-4]','c') = 1 
                                    THEN Total_Charge_Per_Line ELSE 0 END) as "Chiropractic Treatment",
        SUM(CASE WHEN REGEXP_SIMILAR(HCPCS_Line_Procedure_Billed_Code,'9716[1-4]','c') = 1 
                                    THEN Total_Charge_Per_Line ELSE 0 END) as "Physical Therapy",
        SUM(CASE WHEN REGEXP_SIMILAR(HCPCS_Line_Procedure_Billed_Code,'9716[5-8]','c') = 1 
                                    THEN Total_Charge_Per_Line ELSE 0 END) as "Occupational Therapy",
        SUM(CASE WHEN REGEXP_SIMILAR(HCPCS_Line_Procedure_Billed_Code,'970((1[0-9])|(2[0-8]))','c') = 1 
                                    THEN Total_Charge_Per_Line ELSE 0 END) as "Supervised Modalities",
        SUM(CASE WHEN HCPCS_Line_Procedure_Billed_Code in ('63047','63048','63030','63035','22612',
                                                            '22633','22558','22551','64635','64636',
                                                            '63045') 
                                                            THEN Total_Charge_Per_Line ELSE 0 END) as "Spinal Surgery",
        SUM(CASE WHEN HCPCS_Line_Procedure_Billed_Code in ('70551','70553','70544','70547','70549',
                                                           '70540','70543','70540','70543','70336',
                                                           '72141','72156','72146','72157','72148',
                                                           '72158','77059','77021','73221','73223',
                                                           '73721','73723','73718','73720','73218',
                                                           '73220','74181','74183','74185','72195',
                                                           '72197','72195','72197','72195','72197') 
                                    THEN Total_Charge_Per_Line ELSE 0 END) as MRI,
        SUM(CASE WHEN HCPCS_Line_Procedure_Billed_Code in ('72125','72126','72131','72132','72128','72129') 
                                    THEN Total_Charge_Per_Line ELSE 0 END) as "CT Scan",
        SUM(CASE WHEN HCPCS_Line_Procedure_Billed_Code in ('72040','72050','72220','72100','72110','72114',
                                                           '72200','72202','72052','72082','72070')
                                    THEN Total_Charge_Per_Line ELSE 0 END) as "X-ray"
    FROM(
        SELECT 
            (Claim_Administrator_Claim_Number || Insurer_FEIN) as patient_identifier,
            HCPCS_Line_Procedure_Billed_Code, Total_Charge_Per_Line, CAST(Date_of_Bill as CHAR(4)) as DOB
        FROM DEMO_HealthcareFWA.WC_SV1_MSK

        UNION ALL

        SELECT 
            (Claim_Administrator_Claim_Number || Insurer_FEIN) as patient_identifier,
            HCPCS_Line_Procedure_Billed_Code, CAST(Total_Charge_Per_Line as FLOAT), CAST(Date_of_Bill as CHAR(4)) as DOB
        FROM DEMO_HealthcareFWA.WC_SV2_MSK
    ) d
) d
UNPIVOT EXCLUDE NULLS (cost FOR "group" in ("Chiropractic Treatment", "Physical Therapy", "Occupational Therapy",
                                                "Supervised Modalities", "Spinal Surgery", "MRI", "CT Scan", "X-ray")) tmp
) WITH DATA ;    

<p style = 'font-size:16px;font-family:Arial'>In the table below, we are calculating the cost procedure in the musculoskeletal condition.</p>

In [ ]:
CREATE TABLE DEMO_USER.msk_procedures  as (
SELECT
    CAST(Description AS VARCHAR(45)) as "procedure", total_cost
FROM (
    select 
        COALESCE("Sub_Category","SHORT DESCRIPTION") as Description, 
        CAST(SUM(Total_Charge_Per_Line) AS DECIMAL(20,2)) as total_cost
    FROM (
        select
            CAST(Total_Charge_Per_Line AS FLOAT) as Total_Charge_Per_Line,
            HCPCS_Line_Procedure_Billed_Code, CAST(Date_of_Bill as CHAR(4)) as DOB
        from DEMO_HealthcareFWA.WC_SV1_MSK
        WHERE HCPCS_Line_Procedure_Billed_Code is not NULL

        UNION ALL

        select
            CAST(Total_Charge_Per_Line AS FLOAT) as Total_Charge_Per_Line,
            HCPCS_Line_Procedure_Billed_Code, CAST(Date_of_Bill as CHAR(4)) as DOB
        from DEMO_HealthcareFWA.WC_SV2_MSK
        WHERE HCPCS_Line_Procedure_Billed_Code is not NULL
    ) d
    LEFT JOIN DEMO_HealthcareFWA.WC_CPT_Coding_temp
    ON
        (HCPCS_Line_Procedure_Billed_Code BETWEEN "FROM" AND COALESCE("TO","FROM")) AND
            (LENGTH(HCPCS_Line_Procedure_Billed_Code) = LENGTH("FROM"))
    LEFT JOIN DEMO_HealthcareFWA.WC_HCPCS_Coding
    ON HCPCS_Line_Procedure_Billed_Code = HCPC
    GROUP BY Description
) d
QUALIFY RANK() OVER (ORDER BY TOTAL_COST DESC) <= 10
) WITH DATA ;                    

<p style = 'font-size:16px;font-family:Arial'>In the table below, we list the providers of the musculoskeletal conditions.</p>

In [ ]:
CREATE TABLE DEMO_USER.msk_providers  as (
SELECT 
    d.Billing_Provider_FEIN,
    facility_name, Billing_Provider_First_Name, Billing_Provider_Last_Name_or_Group, cost
FROM
(
    SELECT Billing_Provider_FEIN, CAST(SUM(Total_Charge_Per_Line) AS DECIMAL(20,2)) as cost
    FROM (
        select 
            CAST(Total_Charge_Per_Line AS FLOAT) as Total_Charge_Per_Line, Billing_Provider_FEIN,
            CAST(Date_of_Bill as CHAR(4)) as DOB
        from DEMO_HealthcareFWA.WC_SV1_MSK

        UNION ALL

        select
            CAST(Total_Charge_Per_Line AS FLOAT) as Total_Charge_Per_Line, Billing_Provider_FEIN,
            CAST(Date_of_Bill as CHAR(4)) as DOB
        from DEMO_HealthcareFWA.WC_SV2_MSK
    ) d
    GROUP BY Billing_Provider_FEIN
) d
LEFT JOIN DEMO_HealthcareFWA.WC_PROVIDER_LOOKUP lu
ON lu.Billing_Provider_FEIN = d.Billing_Provider_FEIN
QUALIFY RANK() OVER (ORDER BY COST DESC)  <= 20
) WITH DATA;                    

<p style = 'font-size:16px;font-family:Arial'>In the table below, we are calculating total, average, min and max cost and volume of patients having musculoskeletal conditions.</p>

In [ ]:
CREATE TABLE DEMO_USER.msk_drilldown_proc_groups  as (
SELECT
    PRC_GRP as "Procedure Group", 
    CAST(SUM(Total_Charge_Per_Line) AS DECIMAL(20,2)) as "Total Cost",
    SUM(CASE WHEN Total_Charge_Per_Line is NULL THEN 0 ELSE 1 END) as Volume,
    CAST(AVG(Total_Charge_Per_Line) AS DECIMAL(20,2)) as "Average Cost",
    CAST(MIN(Total_Charge_Per_Line) AS DECIMAL(20,2)) as "Min Cost",
    CAST(MAX(Total_Charge_Per_Line) AS DECIMAL(20,2)) as "Max Cost",
    COUNT(DISTINCT CASE WHEN Total_Charge_Per_Line is NULL THEN NULL ELSE patient_identifier END) as "No. of Patients"
FROM DEMO_HealthcareFWA.WC_MSK_PRC_GRP_FILTERED_BILLS
GROUP BY "Procedure Group"
) WITH DATA ;         

<p style = 'font-size:16px;font-family:Arial'>In the table below, we are calculating average cost of procedure per county based on the city provided in the billing information.</p>

In [ ]:
CREATE TABLE DEMO_USER.msk_drilldown_proc_grp_counties  as (
SELECT
    prc_grp as prc_grp,
    county as county, 
    AVG(Total_Charge_Per_Line) as avg_costs
FROM(
    select county, STATE_CODE, HCPCS_Line_Procedure_Billed_Code, Total_Charge_Per_Line
    from DEMO_HealthcareFWA.WC_SV1HEADER hd
    inner join DEMO_HealthcareFWA.WC_SV1DETAIL dt on hd.Bill_ID = dt.Bill_ID
    inner join DEMO_HealthcareFWA.wc_us_cities
    on lower(hd.Billing_Provider_City) = lower(CITY)
    where 
    STATE_CODE = 'TX'
    AND (hd.Date_of_Bill between 20170000 and 20210000)
    AND ((hd.First_ICD9_CM_Diagnosis_Code like 'M__%') OR (hd.Second_ICD9_CM_Diagnosis_Code like 'M__%')
        OR (hd.Third_ICD9_CM_Diagnosis_Code like 'M__%') OR (hd.Fourth_ICD9_CM_Diagnosis_Code like 'M__%'))
) d
INNER JOIN (
  SELECT 
    MIN(Total_Charge_Per_Line) as min_cost,
    MAX(Total_Charge_Per_Line) as max_cost,
    hcpcs_line_procedure_billed_code
  FROM DEMO_HealthcareFWA.WC_MSK_PRC_GRP_FILTERED_BILLS
  GROUP BY hcpcs_line_procedure_billed_code
) b
ON 
  d.HCPCS_Line_Procedure_Billed_Code = b.HCPCS_Line_Procedure_Billed_Code
  AND (d.Total_Charge_Per_Line > b.min_cost AND 
      d.Total_Charge_Per_Line < b.max_cost)
INNER JOIN DEMO_HealthcareFWA.WC_MSK_PRC_GRP
ON REGEXP_SIMILAR(d.HCPCS_Line_Procedure_Billed_Code,CPT_HCPC_CODE) = 1
GROUP BY county,prc_grp
HAVING AVG_COSTS is NOT NULL
    ) WITH DATA ; 

<p style = 'font-size:16px;font-family:Arial'>In the table below, we are calculating total cost and volume of patients having musculoskeletal conditions grouped by procedure and procedure group.</p>

In [ ]:
CREATE TABLE DEMO_USER.msk_drilldown_proc_grp_procedures  as (
SELECT
  prc_grp, proc, cost, volume
FROM (
  select
      prc_grp,
      COALESCE(Description,HCPCS_Line_Procedure_Billed_Code) as proc,
      CAST(SUM(Total_Charge_Per_Line) AS DECIMAL(20,2)) as cost,
      SUM(CASE WHEN Total_Charge_Per_Line is NULL THEN 0 ELSE 1 END) as volume
  FROM DEMO_HealthcareFWA.WC_MSK_PRC_GRP_FILTERED_BILLS d
  LEFT JOIN DEMO_HealthcareFWA.WC_MSK_PRC_DESC
  ON HCPCS_Line_Procedure_Billed_Code = CAST(CPT AS VARCHAR(20))
  GROUP BY prc_grp, proc
) d
) WITH DATA ;       

<p style = 'font-size:16px;font-family:Arial'>Table below gives provider details.</p>

In [ ]:
CREATE TABLE DEMO_USER.msk_drilldown_proc_grp_provs  as (
SELECT 
    PRC_GRP, COUNTY, Billing_Provider_FEIN, Billing_Provider_Last_Name_or_Group,
    AVG(Total_Charge_Per_Line) "Average Cost",
    COUNT(d.HCPCS_Line_Procedure_Billed_Code) volume
FROM(
    select COUNTY, STATE_CODE, HCPCS_Line_Procedure_Billed_Code, Total_Charge_Per_Line, Billing_Provider_FEIN, Billing_Provider_Last_Name_or_Group
    from DEMO_HealthcareFWA.WC_SV1HEADER hd
    inner join DEMO_HealthcareFWA.WC_SV1DETAIL dt on hd.Bill_ID = dt.Bill_ID
    inner join DEMO_HealthcareFWA.wc_us_cities
    on lower(hd.Billing_Provider_City) = lower(CITY)
    where 
    STATE_CODE = 'TX'
    AND (hd.Date_of_Bill between 20170000 and 20210000)
    AND ((hd.First_ICD9_CM_Diagnosis_Code like 'M__%') OR (hd.Second_ICD9_CM_Diagnosis_Code like 'M__%')
        OR (hd.Third_ICD9_CM_Diagnosis_Code like 'M__%') OR (hd.Fourth_ICD9_CM_Diagnosis_Code like 'M__%'))
) d
INNER JOIN (
    SELECT 
        MIN(Total_Charge_Per_Line) as min_cost,
        MAX(Total_Charge_Per_Line) as max_cost,
        hcpcs_line_procedure_billed_code
    FROM DEMO_HealthcareFWA.WC_MSK_PRC_GRP_FILTERED_BILLS
    GROUP BY hcpcs_line_procedure_billed_code
) b
ON 
d.HCPCS_Line_Procedure_Billed_Code = b.HCPCS_Line_Procedure_Billed_Code
AND (d.Total_Charge_Per_Line > b.min_cost AND 
    d.Total_Charge_Per_Line < b.max_cost)
INNER JOIN DEMO_HealthcareFWA.WC_MSK_PRC_GRP
ON REGEXP_SIMILAR(d.HCPCS_Line_Procedure_Billed_Code,CPT_HCPC_CODE) = 1
GROUP BY PRC_GRP, COUNTY, Billing_Provider_FEIN, Billing_Provider_Last_Name_or_Group
) WITH DATA ;         

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial'>4. Reporting tables for Diabetic Conditions</b>
<p style = 'font-size:16px;font-family:Arial'>Here we are creating reporting tables needed for the diabetic condition reporting. First we are creating some driving views which will limit our patient count. We are taking patients which have 90days duration between start and end of encounter.</p>

In [ ]:
Replace VIEW DEMO_USER.va_bk_v_PAT_ENC_LT_90D AS 
SELECT 
    a.PATIENT,
    a.ID ENCOUNTER,
    ENCOUNTERCLASS,
    CODE ENC_CD,
    DESCRIPTION ENC_DESC,
    CAST(SUBSTR("START",0,11) AS DATE FORMAT 'YYYY-MM-DD') STRT_ENC,
    CAST(SUBSTR("STOP",0,11) AS DATE FORMAT 'YYYY-MM-DD') STOP_ENC,
    (STOP_ENC-STRT_ENC) LOS_ENC,
    PROVIDER,
    ORGANIZATION
from DEMO_HealthcareFWA.va_encounters a 
inner join DEMO_HealthcareFWA.va_bk_diab_pat_cnt b
on a.PATIENT = b.PATIENT
WHERE LOS_ENC<90
;

<p style = 'font-size:16px;font-family:Arial'>Next we are calculating the glycated hemoglobin (A1c) values for the patients.</p>

In [ ]:
replace view demo_user.va_bk_v_a1c_enc as
sel b.PATIENT, ENCOUNTER, AVE("VALUE") a1c
from DEMO_HealthcareFWA.va_observations a 
inner join DEMO_HealthcareFWA.va_bk_diab_pat b 
on a.PATIENT=b.PATIENT where  code ='4548-4'
group by 1,2;

<p style = 'font-size:16px;font-family:Arial'>Below is the driving view which holds the codes for diabetic procedures</p>

In [ ]:
REPLACE VIEW demo_user.va_bk_v_diabetes_codes AS 
select distinct code, description, CAST (TRIM(mp.mapTarget) as VARCHAR(15)) Target
    from (
            Select distinct code, description from DEMO_HealthcareFWA.va_conditions
          ) conds
    inner join DEMO_HealthcareFWA.SNOMED_ICD_Map mp
    on conds.code = cast(mp.referencedComponentId as BIGINT)
    where Target like ANY ('V18%', '775%', 'V77%', '648%', '250%', '249%', 'P70%', 'Z83%',
        'O24%', 'Z13%', 'E11%', 'E08%', 'E10%', 'E09%', 'E13%', 'R80%', 'G53%','H54%');


<p style = 'font-size:16px;font-family:Arial'>Below view gives the start date for patient having first diabetic encounter.
</p>

In [ ]:
REPLACE VIEW demo_user.va_bk_diab_strt AS
sel a.PATIENT, MIN("START") STRT_DIAB 
from DEMO_HealthcareFWA.va_bk_diab_pat a 
inner join DEMO_HealthcareFWA.va_conditions b on a. PATIENT=b.PATIENT
inner join demo_user.va_bk_v_diabetes_codes c on b.CODE=c.CODE group by 1

<p style = 'font-size:16px;font-family:Arial'>Below view gives the days and a1c count per year for the patient.</p>

In [ ]:
Replace view demo_user.va_bk_v_a1c_per_yr as
    sel  PATIENT, a1c_per_yr from 
    (
        sel 
            PATIENT, CAST(max_date-min_date as FLOAT) + 1.0 days,
            days/365.0 years, Cast(cnt as FLOAT)/ CEILING(years) a1c_per_yr,
            cnt
        from (
            sel 
                a.PATIENT,
                min(STRT_ENC) min_date,
                max(STRT_ENC) max_date,
                count(a1c) cnt
            from DEMO_USER.va_bk_v_PAT_ENC_LT_90D a 
            inner join DEMO_USER.va_bk_v_a1c_enc b 
            on a.PATIENT=b.PATIENT and a.ENCOUNTER=b.ENCOUNTER
            group by 1
        ) a
    ) a

<p style = 'font-size:16px;font-family:Arial'>Below view holds the overall cost of each encounter the patient had along with its start and end dates.</p>

In [ ]:
REPLACE VIEW DEMO_USER.va_fr_v_ENC_COSTS AS (
    SELECT patient, encounter_id, start_enc, stop_enc, SUM(cost) overall_cost
    FROM(
        SELECT 
            enc.patient as patient, ID as encounter_id,
            CAST(SUBSTR(enc."START",0,11) AS DATE FORMAT 'YYYY-MM-DD') start_enc,
            CAST(SUBSTR(enc.STOP,0,11) AS DATE FORMAT 'YYYY-MM-DD') stop_enc, 
            CAST(total_claim_cost as decimal(18,2)) as enc_cost,
            prc.base_cost as prc_cost,
            imm.base_cost as imm_cost,
            CAST(med.totalcost AS decimal(18,2))as med_cost,
            enc_cost+prc_cost+imm_cost+med_cost as cost
        FROM DEMO_HealthcareFWA.va_encounters enc
        inner join DEMO_HealthcareFWA.va_bk_diab_pat_cnt b
        on enc.PATIENT = b.PATIENT
        LEFT JOIN DEMO_HealthcareFWA.va_procedures prc
        ON enc.ID = prc.encounter
        LEFT JOIN DEMO_HealthcareFWA.va_immunizations imm
        ON enc.ID = imm.encounter
        LEFT JOIN DEMO_HealthcareFWA.va_medications med
        ON enc.ID = med.encounter
    ) d
  GROUP BY patient, encounter_id, start_enc, stop_enc
);

<p style = 'font-size:16px;font-family:Arial'>Below table holds the providers for diabetic conditions. Also the total cost and total and average values of a1c test is also stored for each patient.</p> 

In [ ]:
CREATE TABLE DEMO_USER.dbt_pathwys_prov as (
SELECT 
    h.ID as org_id, h.Name as org_name, 
    SUM(overall_cost) as "Total Cost",
    ZEROIFNULL(AVG(e.cnt)) as "Avg. HBA1C Tests",
    ZEROIFNULL(SUM(e.cnt)) as "Total HBA1C Tests"
    FROM DEMO_USER.va_bk_v_PAT_ENC_LT_90D a
LEFT JOIN (
  sel PATIENT, count(a1c) cnt
  from DEMO_USER.va_bk_v_a1c_enc
  group by 1
) e on a.PATIENT=e.PATIENT
INNER JOIN DEMO_HealthcareFWA.va_organizations h on a.Organization = h.ID
INNER JOIN DEMO_USER.va_fr_v_ENC_COSTS f on a.ENCOUNTER = f.encounter_id
GROUP BY h.ID, h.Name
)WITH DATA ;


<p style = 'font-size:16px;font-family:Arial'>Below table calculate the total procedure costs per group for diabetes procedures.</p>

In [ ]:
CREATE TABLE DEMO_USER.Total_cost_for_procedure_groups_dbt  as (
SELECT "group", cost
FROM(
    SELECT
        SUM(CASE WHEN HCPCS_Line_Procedure_Billed_Code between '82943' and '82962' THEN Total_Charge_Per_Line ELSE 0 END) as "Glucose Testing",
        SUM(CASE WHEN HCPCS_Line_Procedure_Billed_Code between '85004' and '85049' THEN Total_Charge_Per_Line ELSE 0 END) as "Blood Counts",
        SUM(CASE WHEN HCPCS_Line_Procedure_Billed_Code between '36400' and '36425' THEN Total_Charge_Per_Line ELSE 0 END) as "Phlebotomy",
        SUM(CASE WHEN HCPCS_Line_Procedure_Billed_Code between '11042' and '11047' THEN Total_Charge_Per_Line ELSE 0 END) as "Removal of Infected/Devitalized Tissue",
        SUM(CASE WHEN HCPCS_Line_Procedure_Billed_Code between '93000' and '93010' THEN Total_Charge_Per_Line ELSE 0 END) as "Electrocardiogram"
    FROM(
        SELECT 
            (Claim_Administrator_Claim_Number || Insurer_FEIN) as patient_identifier,
            HCPCS_Line_Procedure_Billed_Code, Total_Charge_Per_Line, CAST(Date_of_Bill as CHAR(4)) as DOB
        FROM DEMO_HealthcareFWA.WC_SV1_DBT

        UNION ALL

        SELECT 
            (Claim_Administrator_Claim_Number || Insurer_FEIN) as patient_identifier,
            HCPCS_Line_Procedure_Billed_Code, CAST(Total_Charge_Per_Line as FLOAT), CAST(Date_of_Bill as CHAR(4)) as DOB
        FROM DEMO_HealthcareFWA.WC_SV2_DBT
    ) d
) d
UNPIVOT EXCLUDE NULLS (cost FOR "group" in ("Glucose Testing", "Blood Counts", "Phlebotomy", "Removal of Infected/Devitalized Tissue",
                        "Electrocardiogram")) tmp
    ) WITH DATA ; 

<p style = 'font-size:16px;font-family:Arial'>Below table calculate the total procedure costs for diabetes procedures.</p>

In [ ]:
CREATE TABLE DEMO_USER.Total_cost_for_procedures_dbt  as (
SELECT
    RANK() OVER (ORDER BY TOTAL_COST DESC) as COST_RANK,
    CAST(Description AS VARCHAR(45)) as "Procedure", TOTAL_COST
FROM (
    select 
        COALESCE("Sub_Category","SHORT DESCRIPTION") as Description, 
        CAST(SUM(Total_Charge_Per_Line) AS DECIMAL(20,2)) as TOTAL_COST
    FROM (
        select
            CAST(Total_Charge_Per_Line AS FLOAT) as Total_Charge_Per_Line,
            HCPCS_Line_Procedure_Billed_Code, CAST(Date_of_Bill as CHAR(4)) as DOB
        from DEMO_HealthcareFWA.WC_SV1_DBT
        WHERE HCPCS_Line_Procedure_Billed_Code is not NULL

        UNION ALL

        select
            CAST(Total_Charge_Per_Line AS FLOAT) as Total_Charge_Per_Line,
            HCPCS_Line_Procedure_Billed_Code, CAST(Date_of_Bill as CHAR(4)) as DOB
        from DEMO_HealthcareFWA.WC_SV2_DBT
        WHERE HCPCS_Line_Procedure_Billed_Code is not NULL
    ) d
    LEFT JOIN DEMO_HealthcareFWA.WC_CPT_Coding_temp
    ON
        (HCPCS_Line_Procedure_Billed_Code BETWEEN "FROM" AND COALESCE("TO","FROM")) AND
            (LENGTH(HCPCS_Line_Procedure_Billed_Code) = LENGTH("FROM"))
    LEFT JOIN DEMO_HealthcareFWA.WC_HCPCS_Coding
    ON HCPCS_Line_Procedure_Billed_Code = HCPC
    GROUP BY Description
) d
QUALIFY COST_RANK <= 10
) WITH DATA ;

<p style = 'font-size:16px;font-family:Arial'>Below table calculate and bin the patient volume in various a1c count group.</p>

In [ ]:
CREATE TABLE DEMO_USER.Hemoglobin_count_dbt  as (
SELECT 
    CASE 
        WHEN A1C_COUNT = 1 THEN '1'
        WHEN A1C_COUNT = 2 THEN '2'
        WHEN A1C_COUNT = 3 THEN '3'
        WHEN A1C_COUNT = 4 THEN '4'
        ELSE '5+'
    END AS HBA1C_COUNT, COUNT(patient_identifier) as Patient_Count
FROM (
    SELECT
        d.patient_identifier, COUNT(HCPCS_Line_Procedure_Billed_Code) as A1C_COUNT
    FROM (
        SELECT
            CAST(Total_Charge_Per_Line AS FLOAT) as Total_Charge_Per_Line,
            (Claim_Administrator_Claim_Number || '_' || Insurer_FEIN) as patient_identifier,
            HCPCS_Line_Procedure_Billed_Code, CAST(Date_of_Bill as CHAR(4)) as DOB
        FROM DEMO_HealthcareFWA.WC_SV1HEADER hd
        INNER JOIN DEMO_HealthcareFWA.WC_SV1DETAIL det
        ON hd.Bill_ID = det.Bill_ID
        WHERE HCPCS_Line_Procedure_Billed_Code is not NULL

        UNION ALL

        SELECT
            CAST(Total_Charge_Per_Line AS FLOAT) as Total_Charge_Per_Line,
            (Claim_Administrator_Claim_Number || '_' || Insurer_FEIN) as patient_identifier,
            HCPCS_Line_Procedure_Billed_Code, CAST(Date_of_Bill as CHAR(4)) as DOB
        FROM DEMO_HealthcareFWA.WC_SV2HEADER hd
        INNER JOIN DEMO_HealthcareFWA.WC_SV2DETAIL det
        ON hd.Bill_ID = det.Bill_ID
        WHERE HCPCS_Line_Procedure_Billed_Code is not NULL
    ) d
    INNER JOIN (
        SELECT DISTINCT patient_identifier
        FROM (
            SELECT (Claim_Administrator_Claim_Number || '_' || Insurer_FEIN) as patient_identifier
            FROM DEMO_HealthcareFWA.WC_SV1_DBT

            UNION ALL

            SELECT (Claim_Administrator_Claim_Number || '_' || Insurer_FEIN) as patient_identifier
            FROM DEMO_HealthcareFWA.WC_SV2_DBT            
        ) d
    ) a
    ON d.patient_identifier = a.patient_identifier
    INNER JOIN DEMO_HealthcareFWA.WC_DBT_PRC_DESC
    ON HCPCS_Line_Procedure_Billed_Code = PRC
    WHERE DESCRIPTION = 'Hemoglobin A1c Test'
    GROUP BY d.patient_identifier
) d
GROUP BY HBA1C_COUNT
) WITH DATA ;                  

<p style = 'font-size:16px;font-family:Arial'>Below table gives facilities information.</p>

In [ ]:
CREATE TABLE DEMO_USER.Total_cost_for_facilities_dbt  as (
SELECT 
    RANK() OVER (ORDER BY COST DESC) as COST_RANK, d.Billing_Provider_FEIN,
    Facility_Name, Billing_Provider_First_Name, Billing_Provider_Last_Name_or_Group, COST
FROM
(
    SELECT Billing_Provider_FEIN, CAST(SUM(Total_Charge_Per_Line) AS DECIMAL(20,2)) as COST
    FROM (
        select 
            CAST(Total_Charge_Per_Line AS FLOAT) as Total_Charge_Per_Line, Billing_Provider_FEIN,
            CAST(Date_of_Bill as CHAR(4)) as DOB
        from DEMO_HealthcareFWA.WC_SV1_DBT

        UNION ALL

        select
            CAST(Total_Charge_Per_Line AS FLOAT) as Total_Charge_Per_Line, Billing_Provider_FEIN,
            CAST(Date_of_Bill as CHAR(4)) as DOB
        from DEMO_HealthcareFWA.WC_SV2_DBT
    ) d
    GROUP BY Billing_Provider_FEIN
) d
LEFT JOIN DEMO_HealthcareFWA.WC_PROVIDER_LOOKUP lu
ON lu.Billing_Provider_FEIN = d.Billing_Provider_FEIN
QUALIFY COST_RANK <= 10
) WITH DATA ;                  

<p style = 'font-size:16px;font-family:Arial'>In the table below, we are calculating total cost and volume of patients having diabetes conditions grouped by procedure and procedure group.</p>

In [ ]:
CREATE TABLE DEMO_USER.dbt_drilldown_proc_grp_procedures  as (
SELECT
  PRC_GRP, Proc, Cost, Volume
FROM (
  select
      PRC_GRP,
      COALESCE(Description,HCPCS_Line_Procedure_Billed_Code) as Proc,
      CAST(SUM(Total_Charge_Per_Line) AS DECIMAL(20,2)) as Cost,
      SUM(CASE WHEN Total_Charge_Per_Line is NULL THEN 0 ELSE 1 END) as Volume
  FROM DEMO_HealthcareFWA.WC_DBT_PRC_GRP_FILTERED_BILLS d
  LEFT JOIN DEMO_HealthcareFWA.WC_DBT_PRC_DESC
  ON HCPCS_Line_Procedure_Billed_Code = PRC
  GROUP BY PRC_GRP, Proc
) d
QUALIFY RANK() OVER (PARTITION BY PRC_GRP ORDER BY Cost DESC) <= 30

UNION ALL

SELECT
  DRUG_GRP, Drug, Cost, Volume
FROM (
  select
      DRUG_GRP,
      COALESCE(DESCRIPTION,NDC_Billed_Code) as Drug,
      CAST(SUM(Total_Amount_Paid_Per_Line) AS DECIMAL(20,2)) as Cost,
      SUM(CASE WHEN Total_Amount_Paid_Per_Line is NULL THEN 0 ELSE 1 END) as Volume
  FROM DEMO_HealthcareFWA.WC_DBT_DRUG_GRP_FILTERED_BILLS d
  LEFT JOIN DEMO_HealthcareFWA.WC_DBT_DRUG_DESC
  ON NDC_Billed_Code = NDC_Code
  GROUP BY DRUG_GRP, Drug
) d
QUALIFY RANK() OVER (PARTITION BY DRUG_GRP ORDER BY Cost DESC) <= 30
) WITH DATA ;             

<p style = 'font-size:16px;font-family:Arial'>In the table below, we are calculating total, average, min and max cost and volume of patients having diabetes conditions.</p>


In [ ]:
CREATE TABLE DEMO_USER.dbt_drilldown_proc_groups  as (
SELECT
  PRC_GRP as "Procedure/Treatment Group", 
  CAST(SUM(Total_Charge_Per_Line) AS DECIMAL(20,2)) as "Total Cost",
  SUM(CASE WHEN Total_Charge_Per_Line is NULL THEN 0 ELSE 1 END) as Volume,
  CAST(AVG(Total_Charge_Per_Line) AS DECIMAL(20,2)) as "Average Cost",
  CAST(MIN(Total_Charge_Per_Line) AS DECIMAL(20,2)) as "Min Cost",
  CAST(MAX(Total_Charge_Per_Line) AS DECIMAL(20,2)) as "Max Cost",
  COUNT(DISTINCT CASE WHEN Total_Charge_Per_Line is NULL THEN NULL ELSE patient_identifier END) as "No. of Patients"
FROM DEMO_HealthcareFWA.WC_DBT_PRC_GRP_FILTERED_BILLS
WHERE PRC_GRP <> 'Supervised Modalities'
GROUP BY "Procedure/Treatment Group"

UNION ALL

SELECT
  DRUG_GRP as "Procedure/Treatment Group", 
  CAST(SUM(Total_Amount_Paid_Per_Line) AS DECIMAL(20,2)) as "Total Cost",
  SUM(CASE WHEN Total_Amount_Paid_Per_Line is NULL THEN 0 ELSE 1 END) as Volume,
  CAST(AVG(Total_Amount_Paid_Per_Line) AS DECIMAL(20,2)) as "Average Cost",
  CAST(MIN(Total_Amount_Paid_Per_Line) AS DECIMAL(20,2)) as "Min Cost",
  CAST(MAX(Total_Amount_Paid_Per_Line) AS DECIMAL(20,2)) as "Max Cost",
  COUNT(DISTINCT CASE WHEN Total_Amount_Paid_Per_Line is NULL THEN NULL ELSE patient_identifier END) as "No. of Patients"
FROM DEMO_HealthcareFWA.WC_DBT_DRUG_GRP_FILTERED_BILLS
GROUP BY "Procedure/Treatment Group"
 ) WITH DATA ;      

<p style = 'font-size:16px;font-family:Arial'>In the table below, we are calculating average cost of procedure per county based on the city provided in the billing information.</p>

In [ ]:
CREATE TABLE DEMO_USER.dbt_drilldown_proc_grp_counties  as (
SELECT  PRC_GRP as prc_grp,
    county, 
    AVG(Total_Charge_Per_Line) as avg_costs
FROM(
    select COUNTY as county, STATE_CODE, HCPCS_Line_Procedure_Billed_Code, Total_Charge_Per_Line
    from DEMO_HealthcareFWA.WC_SV1HEADER hd
    inner join DEMO_HealthcareFWA.WC_SV1DETAIL dt on hd.Bill_ID = dt.Bill_ID
    inner join DEMO_HealthcareFWA.wc_us_cities
    on lower(hd.Billing_Provider_City) = lower(CITY)
    INNER JOIN (
        SELECT DISTINCT Claim_Administrator_Claim_Number, Insurer_FEIN
        FROM (
            SELECT Claim_Administrator_Claim_Number, Insurer_FEIN
            FROM DEMO_HealthcareFWA.WC_SV1_DBT

            UNION ALL

            SELECT Claim_Administrator_Claim_Number, Insurer_FEIN
            FROM DEMO_HealthcareFWA.WC_SV2_DBT            
        ) v
    ) a
    ON 
        hd.Claim_Administrator_Claim_Number = a.Claim_Administrator_Claim_Number AND 
        hd.Insurer_FEIN = a.Insurer_FEIN
        where STATE_CODE = 'TX' AND (hd.Date_of_Bill between 20170000 and 20210000)
) d
INNER JOIN (
    SELECT 
    MIN(Total_Charge_Per_Line) as min_cost,
    MAX(Total_Charge_Per_Line) as max_cost,
    hcpcs_line_procedure_billed_code
    FROM DEMO_HealthcareFWA.WC_DBT_PRC_GRP_FILTERED_BILLS
    GROUP BY hcpcs_line_procedure_billed_code
) b
ON 
    d.HCPCS_Line_Procedure_Billed_Code = b.HCPCS_Line_Procedure_Billed_Code
    AND (d.Total_Charge_Per_Line > b.min_cost AND 
        d.Total_Charge_Per_Line < b.max_cost)
INNER JOIN DEMO_HealthcareFWA.WC_DBT_PRC_GRP
ON REGEXP_SIMILAR(d.HCPCS_Line_Procedure_Billed_Code,CPT_HCPC_CODE) = 1
GROUP BY COUNTY,PRC_GRP
HAVING AVG_COSTS is NOT NULL

UNION ALL

SELECT  DRUG_GRP as prc_grp,
  county, 
  AVG(Total_Amount_Paid_Per_Line) as avg_costs
FROM(
  SELECT COUNTY as county, STATE_CODE, NDC_Billed_Code, Total_Amount_Paid_Per_Line
  FROM DEMO_HealthcareFWA.WC_SV4HEADER hd
  INNER JOIN DEMO_HealthcareFWA.WC_SV4DETAIL dt 
  ON hd.Bill_ID = dt.Bill_ID
  INNER JOIN wc_us_cities
  ON lower(hd.Billing_Provider_City) = lower(CITY)
  INNER JOIN (
      SELECT DISTINCT Claim_Administrator_Claim_Number, Insurer_FEIN
      FROM (
          SELECT Claim_Administrator_Claim_Number, Insurer_FEIN
          FROM DEMO_HealthcareFWA.WC_SV1_DBT

          UNION ALL

          SELECT Claim_Administrator_Claim_Number, Insurer_FEIN
          FROM DEMO_HealthcareFWA.WC_SV2_DBT            
      ) v
  ) a
  ON 
      hd.Claim_Administrator_Claim_Number = a.Claim_Administrator_Claim_Number AND 
      hd.Insurer_FEIN = a.Insurer_FEIN
      where STATE_CODE = 'TX'
) d
INNER JOIN (
  SELECT 
      MIN(Total_Amount_Paid_Per_Line) as min_cost,
      MAX(Total_Amount_Paid_Per_Line) as max_cost,
      ndc_billed_code
  FROM DEMO_HealthcareFWA.WC_DBT_DRUG_GRP_FILTERED_BILLS
  GROUP BY ndc_billed_code
) b
ON 
  d.ndc_billed_code = b.ndc_billed_code
  AND (d.Total_Amount_Paid_Per_Line > b.min_cost AND 
      d.Total_Amount_Paid_Per_Line < b.max_cost)
INNER JOIN DEMO_HealthcareFWA.WC_DBT_DRUG_DESC dsc
ON d.NDC_Billed_Code = dsc.NDC_Code
INNER JOIN DEMO_HealthcareFWA.WC_DBT_DRUG_GRP grp
ON dsc.DESCRIPTION = grp.DESCRIPTION
GROUP BY COUNTY,DRUG_GRP
HAVING AVG_COSTS is NOT NULL
) WITH DATA ;        

<p style = 'font-size:16px;font-family:Arial'>Table below gives provider details.</p>

In [ ]:
CREATE TABLE DEMO_USER.dbt_drilldown_proc_grp_provs  as (
SELECT 
                    PRC_GRP, COUNTY,Billing_Provider_FEIN, Billing_Provider_Last_Name_or_Group, AVG(Total_Charge_Per_Line) "Average Cost", COUNT(d.HCPCS_Line_Procedure_Billed_Code) volume
                    FROM(
                        SELECT COUNTY, STATE_CODE, HCPCS_Line_Procedure_Billed_Code, Total_Charge_Per_Line, Billing_Provider_FEIN, Billing_Provider_Last_Name_or_Group
                        FROM DEMO_HealthcareFWA.WC_SV1HEADER hd
                        INNER JOIN DEMO_HealthcareFWA.WC_SV1DETAIL dt 
                        ON hd.Bill_ID = dt.Bill_ID
                        INNER JOIN DEMO_HealthcareFWA.wc_us_cities
                        ON lower(hd.Billing_Provider_City) = lower(CITY)
                        INNER JOIN (
                            SELECT DISTINCT Claim_Administrator_Claim_Number, Insurer_FEIN
                            FROM (
                                SELECT Claim_Administrator_Claim_Number, Insurer_FEIN
                                FROM DEMO_HealthcareFWA.WC_SV1_DBT
                    
                                UNION ALL
                    
                                SELECT Claim_Administrator_Claim_Number, Insurer_FEIN
                                FROM DEMO_HealthcareFWA.WC_SV2_DBT
                            ) v
                        ) a
                        ON 
                            hd.Claim_Administrator_Claim_Number = a.Claim_Administrator_Claim_Number AND 
                            hd.Insurer_FEIN = a.Insurer_FEIN
                        WHERE STATE_CODE = 'TX'
                    ) d
                    INNER JOIN (
                        SELECT 
                            MIN(Total_Charge_Per_Line) as min_cost,
                            MAX(Total_Charge_Per_Line) as max_cost,
                            hcpcs_line_procedure_billed_code
                        FROM DEMO_HealthcareFWA.WC_DBT_PRC_GRP_FILTERED_BILLS
                        GROUP BY hcpcs_line_procedure_billed_code
                    ) b
                    ON 
                    d.HCPCS_Line_Procedure_Billed_Code = b.HCPCS_Line_Procedure_Billed_Code
                    AND (d.Total_Charge_Per_Line > b.min_cost AND 
                        d.Total_Charge_Per_Line < b.max_cost)
                    INNER JOIN DEMO_HealthcareFWA.WC_DBT_PRC_GRP
                    ON REGEXP_SIMILAR(d.HCPCS_Line_Procedure_Billed_Code,CPT_HCPC_CODE) = 1
                    GROUP BY Billing_Provider_FEIN, Billing_Provider_Last_Name_or_Group,PRC_GRP,COUNTY
                    
                    UNION ALL     
                         
                    SELECT
                        DRUG_GRP as PRC_GRP, COUNTY,Billing_Provider_FEIN, Billing_Provider_Last_Name_or_Group,
                        AVG(Total_Amount_Paid_Per_Line) "Average Cost",
                        COUNT(d.NDC_Billed_Code) volume
                    FROM(
                        SELECT COUNTY, STATE_CODE, NDC_Billed_Code, Total_Amount_Paid_Per_Line, Billing_Provider_FEIN, Billing_Provider_Last_Name_or_Group
                        FROM DEMO_HealthcareFWA.WC_SV4HEADER hd
                        INNER JOIN DEMO_HealthcareFWA.WC_SV4DETAIL dt 
                        ON hd.Bill_ID = dt.Bill_ID
                        INNER JOIN DEMO_HealthcareFWA.wc_us_cities
                        ON lower(hd.Billing_Provider_City) = lower(CITY)
                        INNER JOIN (
                            SELECT DISTINCT Claim_Administrator_Claim_Number, Insurer_FEIN
                            FROM (
                                SELECT Claim_Administrator_Claim_Number, Insurer_FEIN
                                FROM DEMO_HealthcareFWA.WC_SV1_DBT
                    
                                UNION ALL
                    
                                SELECT Claim_Administrator_Claim_Number, Insurer_FEIN
                                FROM DEMO_HealthcareFWA.WC_SV2_DBT
                            ) v
                        ) a
                        ON 
                            hd.Claim_Administrator_Claim_Number = a.Claim_Administrator_Claim_Number AND 
                            hd.Insurer_FEIN = a.Insurer_FEIN
                        WHERE STATE_CODE = 'TX'
                    ) d
                    INNER JOIN (
                        SELECT 
                            MIN(Total_Amount_Paid_Per_Line) as min_cost,
                            MAX(Total_Amount_Paid_Per_Line) as max_cost,
                            ndc_billed_code
                        FROM DEMO_HealthcareFWA.WC_DBT_DRUG_GRP_FILTERED_BILLS
                        GROUP BY ndc_billed_code
                    ) b
                    ON 
                        d.ndc_billed_code = b.ndc_billed_code
                        AND (d.Total_Amount_Paid_Per_Line > b.min_cost AND 
                            d.Total_Amount_Paid_Per_Line < b.max_cost)
                    
                    INNER JOIN DEMO_HealthcareFWA.WC_DBT_DRUG_DESC dsc
                    ON d.NDC_Billed_Code = dsc.NDC_Code
                    INNER JOIN DEMO_HealthcareFWA.WC_DBT_DRUG_GRP grp
                    ON dsc.DESCRIPTION = grp.DESCRIPTION
                    GROUP BY Billing_Provider_FEIN, Billing_Provider_Last_Name_or_Group, DRUG_GRP,COUNTY
  ) WITH DATA ;     

<p style = 'font-size:16px;font-family:Arial'>Below view list the patient and condition start date.</p>

In [ ]:
REPLACE VIEW demo_user.va_bk_v_sel_hrt_rnl AS  
    sel distinct 
        PATIENT, CODE COND_CD, encounter, CAST(SUBSTR("START",0,11) AS DATE FORMAT 'YYYY-MM-DD') COND_DT,
        DESCRIPTION COND  
    from DEMO_HealthcareFWA.va_conditions 
    where code in (433144002,88805009,53741008,84114007,127013003)

<p style = 'font-size:16px;font-family:Arial'>In the table below we are calculating the path of procedures taken by a diabetic patient with cost of each procedure , a1c count as well as overall cost of all procedures.</p>

In [ ]:
CREATE TABLE DEMO_USER.dbt_pathways  as (
SELECT 
  TRIM (BOTH FROM REGEXP_REPLACE(patient, '[\[\]]', '', 1, 0, 'c')) as patient_identifier,
  COND || ', '|| REGEXP_REPLACE(path, '[\[\]]+', '', 1, 0, 'c') AS path,
  cond, cond_dt,
  last_enc,
  a1c_val, a1c_per_yr,                               
  (LAST_ENC-COND_DT)/365 num_yrs,
  length,
  '0,' || REGEXP_REPLACE(path_cost, '[\[\]\s]+', '', 1, 0, 'c') as path_cost,
  'Start,' || REGEXP_REPLACE(provider_id, '[\[\]\s]+', '', 1, 0, 'c') AS provider_id
FROM nPath (
ON (      
    sel 
      a.PATIENT, ENCOUNTERCLASS, ENC_DESC, STRT_ENC,
      COND, COND_DT, STRT_DIAB,
      LAST_VALUE(STRT_ENC) OVER (PARTITION BY a.PATIENT ORDER BY STRT_ENC) LST_ENC,
      COALESCE(a1c, 0.0) a1c_val, COALESCE(a1c_per_yr, 0.0) a1c_per_yr, overall_cost,
      h.Name as prov_org, h.ID as prov_org_id
      from demo_user.va_bk_v_PAT_ENC_LT_90D a
    inner join (
      sel * from DEMO_HealthcareFWA.va_bk_diab_pat_cnt
      ) b on a.PATIENT=b.PATIENT
    inner join DEMO_user.va_bk_diab_strt c on a.PATIENT=c.PATIENT
    inner join DEMO_user.va_bk_v_sel_hrt_rnl x on a.PATIENT=x.PATIENT
    left join DEMO_user.va_bk_v_a1c_enc d on a.PATIENT=d.PATIENT and a.ENCOUNTER=d.ENCOUNTER
    inner join DEMO_user.va_bk_v_a1c_per_yr e on a.PATIENT=e.PATIENT
    inner join DEMO_user.va_fr_v_ENC_COSTS f on a.ENCOUNTER = f.encounter_id
    inner join DEMO_HealthcareFWA.va_providers g on a.PROVIDER = g.ID
    inner join DEMO_HealthcareFWA.va_organizations h on g.Organization = h.ID
        where 
      lower(ENCOUNTERCLASS) not like all ('wellness', 'ambulatory')
      AND STRT_ENC < ADD_MONTHS(COND_DT, 5*12)
)
PARTITION BY a.PATIENT, COND
ORDER BY STRT_ENC
USING
Mode (nonoverlapping)
Pattern ('A+')
Symbols (
    STRT_ENC > COND_DT AS A
)
Result (
    
    FIRST (PATIENT OF A) AS patient,
    LAST (STRT_ENC OF A) AS last_enc,
    FIRST (COND_DT OF A) AS cond_dt,
    FIRST (COND OF A) AS cond, 
    AVE (a1c_val OF A) AS a1c_val,
    FIRST (a1c_per_yr OF A) AS a1c_per_yr,
    ACCUMULATE (CAST (ENC_DESC AS VARCHAR(40)) OF A) AS path,
    COUNT(* of A) as length,
    ACCUMULATE(overall_cost OF A) as path_cost,
    ACCUMULATE(prov_org_id OF A) as provider_id
)
) p
    ) WITH DATA; 

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial'>5. Reporting tables for Overall Summary</b>
<p style = 'font-size:16px;font-family:Arial'>Here we are creating reporting tables needed for the overall reporting. The table below list the cost of each chapter i.e MSK(musculoskeletal), DBT (Diabetic) and ALL.</p>

In [ ]:
CREATE TABLE DEMO_USER.chapter_summary as (
select 
    COUNT(DISTINCT patient_identifier) as patients,
    SUM(Total_Charge_Per_Line) as costs,
    SUM(CASE WHEN REGEXP_SIMILAR(HCPCS_Line_Procedure_Billed_Code,'9716[1-4]','c') = 1 THEN 1 ELSE 0 END) pt_visits,
    SUM(CASE WHEN REGEXP_SIMILAR(HCPCS_Line_Procedure_Billed_Code,'9894[0-4]','c') = 1 THEN 1 ELSE 0 END) chiro_trts,
    'MSK' as label
from (
    SELECT 
        (Claim_Administrator_Claim_Number || '_' || Insurer_FEIN) as patient_identifier, Total_Charge_Per_Line,
        HCPCS_Line_Procedure_Billed_Code
    FROM DEMO_HealthcareFWA.WC_SV1_MSK
    UNION ALL
    SELECT 
        (Claim_Administrator_Claim_Number || '_' || Insurer_FEIN) as patient_identifier, CAST(Total_Charge_Per_Line AS FLOAT),
        HCPCS_Line_Procedure_Billed_Code
    FROM DEMO_HealthcareFWA.WC_SV2_MSK 
) d

UNION ALL

select 
    COUNT(DISTINCT patient_identifier) as patients,
    SUM(Total_Charge_Per_Line) as costs,
    SUM(CASE WHEN REGEXP_SIMILAR(HCPCS_Line_Procedure_Billed_Code,'9716[1-4]','c') = 1 THEN 1 ELSE 0 END) pt_visits,
    SUM(CASE WHEN REGEXP_SIMILAR(HCPCS_Line_Procedure_Billed_Code,'9894[0-4]','c') = 1 THEN 1 ELSE 0 END) chiro_trts,
    'DBT' as label
from (
    SELECT 
        (Claim_Administrator_Claim_Number || '_' || Insurer_FEIN) as patient_identifier, Total_Charge_Per_Line,
        HCPCS_Line_Procedure_Billed_Code
    FROM DEMO_HealthcareFWA.WC_SV1_DBT
    
    UNION ALL
    
    SELECT 
        (Claim_Administrator_Claim_Number || '_' || Insurer_FEIN) as patient_identifier, CAST(Total_Charge_Per_Line AS FLOAT),
        HCPCS_Line_Procedure_Billed_Code
    FROM DEMO_HealthcareFWA.WC_SV2_DBT 
) db


UNION ALL

select 
    COUNT(DISTINCT patient_identifier) as PATIENTS,
    SUM(Total_Charge_Per_Line) as COSTS,
    SUM(CASE WHEN REGEXP_SIMILAR(HCPCS_Line_Procedure_Billed_Code,'9716[1-4]','c') = 1 THEN 1 ELSE 0 END) pt_visits,
    SUM(CASE WHEN REGEXP_SIMILAR(HCPCS_Line_Procedure_Billed_Code,'9894[0-4]','c') = 1 THEN 1 ELSE 0 END) chiro_trts,
    'ALL'
from (
    SELECT 
        (Claim_Administrator_Claim_Number || '_' || Insurer_FEIN) as patient_identifier, Total_Charge_Per_Line,
        HCPCS_Line_Procedure_Billed_Code
    FROM DEMO_HealthcareFWA.WC_SV1HEADER head
    INNER JOIN DEMO_HealthcareFWA.WC_SV1DETAIL det
    on head.Bill_ID = det.Bill_ID
    WHERE Date_of_Bill between 20170000 and 20210000
    
    UNION ALL
    
    SELECT 
        (Claim_Administrator_Claim_Number || '_' || Insurer_FEIN) as patient_identifier, CAST(Total_Charge_Per_Line AS FLOAT),
        HCPCS_Line_Procedure_Billed_Code
    FROM DEMO_HealthcareFWA.WC_SV2HEADER head
    INNER JOIN DEMO_HealthcareFWA.WC_SV2DETAIL det
    on head.Bill_ID = det.Bill_ID
    WHERE Date_of_Bill between 20170000 and 20210000
) d
) WITH DATA;   

<p style = 'font-size:16px;font-family:Arial'>Below table list the volume and cost of procedures calculated from billing data.</p>

In [ ]:
CREATE TABLE DEMO_USER.overview_chapters  as (
SELECT
    RANK() OVER (ORDER BY COST DESC) as cost_rank,
    chapter,
    cost,
    volume
FROM (
    select 
        DESCRIPTION as chapter,
        CAST(SUM(Total_Charge_Per_Line) AS DECIMAL(20,2)) as cost,
        COUNT(*) AS volume
    FROM (
        select
            CAST(Total_Charge_Per_Line AS FLOAT) as Total_Charge_Per_Line,
            First_ICD9_CM_Diagnosis_Code, CAST(Date_of_Bill as CHAR(4)) as DOB
        from DEMO_HealthcareFWA.WC_SV1HEADER hd
        INNER JOIN DEMO_HealthcareFWA.WC_SV1DETAIL det
        ON hd.Bill_ID = det.Bill_ID
        WHERE First_ICD9_CM_Diagnosis_Code is not NULL 

        UNION ALL

        select
            CAST(Total_Charge_Per_Line AS FLOAT) as Total_Charge_Per_Line,
            First_ICD9_CM_Diagnosis_Code, CAST(Date_of_Bill as CHAR(4)) as DOB
        from DEMO_HealthcareFWA.WC_SV2HEADER hd
        INNER JOIN DEMO_HealthcareFWA.WC_SV2DETAIL det
        ON hd.Bill_ID = det.Bill_ID
        WHERE First_ICD9_CM_Diagnosis_Code is not NULL 
    ) d
    INNER JOIN DEMO_HealthcareFWA.WC_MSK_DIAG_CHP
    ON CAST(First_ICD9_CM_Diagnosis_Code AS CHAR(3)) BETWEEN D_FROM and D_TO
    GROUP BY chapter
) d
WHERE 
    chapter not like 'Factors Influencing Health Status' AND 
    chapter not like 'Symptoms, Signs and Clinical Findings'
QUALIFY cost_rank <= 5
        ) WITH DATA ;   

<p style = 'font-size:16px;font-family:Arial'>Below table list the information of all the procedures.</p>

In [ ]:
CREATE TABLE DEMO_USER.overview_procedures   as (
SELECT
    RANK() OVER (ORDER BY TOTAL_COST DESC) as cost_rank,
    CAST("procedure" AS VARCHAR(45)) as "procedure", total_cost
FROM (
    select 
        COALESCE("Sub_Category","SHORT DESCRIPTION") as "procedure", 
        CAST(SUM(Total_Charge_Per_Line) AS DECIMAL(20,2)) as total_cost
    FROM (
        select
            CAST(Total_Charge_Per_Line AS FLOAT) as Total_Charge_Per_Line,
            HCPCS_Line_Procedure_Billed_Code, CAST(Date_of_Bill as CHAR(4)) as DOB
        from DEMO_HealthcareFWA.WC_SV1HEADER hd
        INNER JOIN DEMO_HealthcareFWA.WC_SV1DETAIL det
        ON hd.Bill_ID = det.Bill_ID
        WHERE HCPCS_Line_Procedure_Billed_Code is not NULL 

        UNION ALL

        select
            CAST(Total_Charge_Per_Line AS FLOAT) as Total_Charge_Per_Line,
            HCPCS_Line_Procedure_Billed_Code, CAST(Date_of_Bill as CHAR(4)) as DOB
        from DEMO_HealthcareFWA.WC_SV2HEADER hd
        INNER JOIN DEMO_HealthcareFWA.WC_SV2DETAIL det
        ON hd.Bill_ID = det.Bill_ID
        WHERE HCPCS_Line_Procedure_Billed_Code is not NULL 
    ) d
    LEFT JOIN DEMO_HealthcareFWA.WC_CPT_Coding_temp
    ON
        (HCPCS_Line_Procedure_Billed_Code BETWEEN "FROM" AND COALESCE("TO","FROM")) AND
            (LENGTH(HCPCS_Line_Procedure_Billed_Code) = LENGTH("FROM"))
    LEFT JOIN DEMO_HealthcareFWA.WC_HCPCS_Coding
    ON HCPCS_Line_Procedure_Billed_Code = HCPC
    GROUP BY "procedure"
) d
QUALIFY cost_rank <= 20
 ) WITH DATA ;                  

<p style = 'font-size:16px;font-family:Arial'>Below table list the provider details.</p>

In [ ]:
CREATE TABLE DEMO_USER.overview_providers  as (
SELECT
    RANK() OVER (ORDER BY COST DESC) as cost_rank, d.Billing_Provider_FEIN as provider_fein,
    coalesce(Facility_Name,Billing_Provider_Last_Name_or_Group) as facility_name,
    Billing_Provider_First_Name as provider_first_name, Billing_Provider_Last_Name_or_Group as provider_group, cost
FROM
(
    SELECT 
          Billing_Provider_FEIN,
          CAST(AVG(Total_Charge_Per_Line) AS DECIMAL(20,2)) as cost
    FROM (
        select
            CAST(Total_Charge_Per_Line AS FLOAT) as Total_Charge_Per_Line,
            Billing_Provider_FEIN, CAST(Date_of_Bill as CHAR(4)) as DOB
            from DEMO_HealthcareFWA.WC_SV1HEADER hd
            INNER JOIN DEMO_HealthcareFWA.WC_SV1DETAIL det
            ON hd.Bill_ID = det.Bill_ID
            WHERE (Date_of_Bill between 20170000 and 20210000)

        UNION ALL

        select
            CAST(Total_Charge_Per_Line AS FLOAT) as Total_Charge_Per_Line, Billing_Provider_FEIN,
            CAST(Date_of_Bill as CHAR(4)) as DOB
            from DEMO_HealthcareFWA.WC_SV2HEADER hd
            INNER JOIN DEMO_HealthcareFWA.WC_SV2DETAIL det
            ON hd.Bill_ID = det.Bill_ID
            WHERE (Date_of_Bill between 20170000 and 20210000)
    ) d
    GROUP BY Billing_Provider_FEIN
) d
LEFT JOIN DEMO_HealthcareFWA.WC_PROVIDER_LOOKUP lu
ON lu.Billing_Provider_FEIN = d.Billing_Provider_FEIN
QUALIFY COST_RANK <= 20
 ) WITH DATA ;                

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial'>5. Reporting tables for Maternity Conditions</b>
<p style = 'font-size:16px;font-family:Arial'>Here we are creating reporting tables needed for the maternity condition reporting. Below table lists the infomation on the procedure groups.</p>

In [ ]:
CREATE TABLE DEMO_USER.mat_proc_grps  as (
SELECT
    CASE 
        WHEN (chapter in ('O82', 'Z38.01', 'Z38,62', 'Z38.64', 'Z38.66', 'Z38.31', 'Z38.69','740') OR 
            HCPCS_Line_Procedure_Billed_Code in ('59510','59514','59515')) OR 
            ICD9_CM_Principal_Procedure_Code in ('740','741','742','744','749','7499') THEN 'Cesarean Delivery'
        WHEN HCPCS_Line_Procedure_Billed_Code in ('99291','99292') then 'Neonatal Critical Care'
        WHEN HCPCS_Line_Procedure_Billed_Code in ('82105') then 'Alpha Fetoprotein'
    END as treatment, SUM(Total_Charge_Per_Line) total_cost, COUNT(DISTINCT patient_identifier) patients, (total_cost/patients) avg_cost
FROM (
    SELECT *
    FROM (
        select
            (Claim_Administrator_Claim_Number || Insurer_FEIN) as patient_identifier, HCPCS_Line_Procedure_Billed_Code, 'NULL' as ICD9_CM_Principal_Procedure_Code,
            First_ICD9_CM_Diagnosis_Code as chp_1, Second_ICD9_CM_Diagnosis_Code as chp_2,
            Third_ICD9_CM_Diagnosis_Code as chp_3, Fourth_ICD9_CM_Diagnosis_Code as chp_4,
            Fifth_ICD9_CM_Diagnosis_Code as chp_5, Total_Charge_Per_Line
        from DEMO_HealthcareFWA.WC_SV1_MAT

        UNION ALL

   --Maternity Procedure Groups     
        select
            (Claim_Administrator_Claim_Number || Insurer_FEIN) as patient_identifier, HCPCS_Line_Procedure_Billed_Code, ICD9_CM_Principal_Procedure_Code,
            First_ICD9_CM_Diagnosis_Code as chp_1, Second_ICD9_CM_Diagnosis_Code as chp_2,
            Third_ICD9_CM_Diagnosis_Code as chp_3, Fourth_ICD_9_CM_Diagnosis_Code as chp_4,
            Fifth_ICD9_CM_Diagnosis_Code as chp_5, CAST(Total_Charge_Per_Line AS FLOAT)
        from DEMO_HealthcareFWA.WC_SV2_MAT

    ) d
    UNPIVOT EXCLUDE NULLS
    (
        chapter FOR diag_level IN
        (chp_1, chp_2, chp_3, chp_4, chp_5)
    ) Tmp
) up
Where treatment is not NULL
GROUP BY treatment
) WITH DATA ;                  ;

<p style = 'font-size:16px;font-family:Arial'>Below view is the driving view for the patients with maternity condition.</p>

In [ ]:
REPLACE VIEW DEMO_USER.WC_FR_V_NPATH_PRG_QRY AS (
    SELECT
        (Claim_Administrator_Claim_Number ||'_'|| Insurer_FEIN) as patient_identifier,
        HCPCS_Line_Procedure_Billed_Code, Total_Charge_Per_Line,
        Service_Line_From_Date,
        min(Service_Line_From_Date) OVER (PARTITION BY patient_identifier) first_mat_diag,
        COUNT(Service_Line_From_Date) OVER (PARTITION BY patient_identifier) proc_count,
        Billing_Provider_Last_Name_or_Group,
        Billing_Provider_FEIN,
        First_ICD9_CM_Diagnosis_Code, Second_ICD9_CM_Diagnosis_Code, Third_ICD9_CM_Diagnosis_Code,
        Fourth_ICD9_CM_Diagnosis_Code, Fifth_ICD9_CM_Diagnosis_Code,
        ICD9_CM_Principal_Procedure_Code,
        COALESCE(Description,"Sub_Category") as PRC_DESC, -- ,""SHORT DESCRIPTION""
        COALESCE(ch_ds.descr,CAST(First_ICD9_CM_Diagnosis_Code as CHAR(3))) primary_condition,
        Procedure_Description,
        CASE
          WHEN (First_ICD9_CM_Diagnosis_Code in 
                  ('O82', 'Z38.01', 'Z38,62', 'Z38.64', 'Z38.66', 'Z38.31', 'Z38.69','740') OR 
                HCPCS_Line_Procedure_Billed_Code in ('59510','59514','59515')) OR 
                ICD9_CM_Principal_Procedure_Code in ('740','741','742','744','749','7499') THEN 'Cesarean Delivery'
          WHEN HCPCS_Line_Procedure_Billed_Code in ('99291','99292') then 'Neonatal Critical Care'
          WHEN HCPCS_Line_Procedure_Billed_Code in ('82105','82106') then 'Alpha Fetoprotein'
          WHEN REGEXP_SIMILAR(HCPCS_Line_Procedure_Billed_Code,'9894[0-4]','c') = 1 
              THEN 'Chiropractic Manipulative Treatment'
          WHEN REGEXP_SIMILAR(HCPCS_Line_Procedure_Billed_Code,'9716[1-4]','c') = 1
              THEN 'Physical Therapy'
          WHEN REGEXP_SIMILAR(HCPCS_Line_Procedure_Billed_Code,'9716[5-8]','c') = 1
              THEN 'Occupational Therapy'
          WHEN REGEXP_SIMILAR(HCPCS_Line_Procedure_Billed_Code,'970((1[0-9])|(2[0-8]))','c') = 1
              THEN 'Supervised Modalities'
          WHEN HCPCS_Line_Procedure_Billed_Code in ('63047', '63048', '63030', '63035',
                                                    '22612', '22633', '22558', '22551',
                                                    '64635', '64636', '63045') THEN 'Spinal Surgery'
          WHEN HCPCS_Line_Procedure_Billed_Code in ('70551','70553','70544','70547',
                                                    '70549','70540','70543','70540',
                                                    '70543','70336') THEN 'Back/Neck MRI'
          WHEN HCPCS_Line_Procedure_Billed_Code in ('72141','72156','72146','72157',
                                                     '72148','72158') THEN 'Spine MRI'
          WHEN HCPCS_Line_Procedure_Billed_Code in ('77059','77021') THEN 'Breast MRI'
          WHEN HCPCS_Line_Procedure_Billed_Code in ('73221','73223','73721','73723') THEN 'Joints MRI'
          WHEN HCPCS_Line_Procedure_Billed_Code in ('73718','73720','73218','73220') THEN 'Extremity MRI'
          WHEN HCPCS_Line_Procedure_Billed_Code in ('74181','74183','74185') THEN 'Abdomen MRI'
          WHEN HCPCS_Line_Procedure_Billed_Code in ('72195','72197','72195',
                                                    '72197','72195','72197') THEN 'Pelvis MRI'
          WHEN HCPCS_Line_Procedure_Billed_Code in ('72125','72126','72131',
                                                    '72132''72128','72129') THEN 'CT Scan for Spine'
          WHEN HCPCS_Line_Procedure_Billed_Code in ('72040','72050','72220',
                                                    '72100','72110','72114',
                                                    '72200','72202','72052',
                                                    '72082','72070') THEN 'Xray for Spine'
          ELSE NULL END as Description,
        Employee_Mailing_City
    FROM DEMO_HealthcareFWA.WC_MAT_PATIENT_ALL_BILLS
    LEFT JOIN DEMO_HealthcareFWA.WC_CHP_DESC ch_ds
    ON CAST(First_ICD9_CM_Diagnosis_Code as CHAR(3)) = CAST(ch_ds.code as CHAR(3))
    LEFT JOIN DEMO_HealthcareFWA.WC_CPT_Coding_temp
    ON
        (HCPCS_Line_Procedure_Billed_Code BETWEEN "FROM" AND COALESCE("TO","FROM")) AND
            (LENGTH(HCPCS_Line_Procedure_Billed_Code) = LENGTH("FROM"))
    LEFT JOIN DEMO_HealthcareFWA.WC_HCPCS_Coding
    ON HCPCS_Line_Procedure_Billed_Code = HCPC
    WHERE 
        HCPCS_Line_Procedure_Billed_Code is not null and First_ICD9_CM_Diagnosis_Code is not null
        and PRC_DESC is not null
)

<p style = 'font-size:16px;font-family:Arial'> Below table lists the pathways in maternity condition.</p>

In [ ]:
CREATE TABLE Demo_User.mat_pathways  as (
SELECT
TRIM (BOTH FROM REGEXP_REPLACE(patient_identifier, '[\[\]]', '', 1, 0, 'c')) as patient_identifier,
'Pregnancy Condition,' || REGEXP_REPLACE(path, '[\[\]]+', '', 1, 0, 'c') AS path,
'0.0,' || REGEXP_REPLACE(cost, '[\[\]\s]+', '', 1, 0, 'c') AS cost,
label,
'None,' || REGEXP_REPLACE(prov_group, '[\[\]]+', '', 1, 0, 'c') as prov_group,
'None,' || REGEXP_REPLACE(provider, '[\[\]\s]+', '', 1, 0, 'c') as provider,
total_cost
FROM nPath (
ON (
  SELECT
      a.patient_identifier, HCPCS_Line_Procedure_Billed_Code, Service_Line_From_Date, first_mat_diag,
      Billing_Provider_Last_Name_or_Group, Billing_Provider_FEIN,
      First_ICD9_CM_Diagnosis_Code, Second_ICD9_CM_Diagnosis_Code, Third_ICD9_CM_Diagnosis_Code,
      Fourth_ICD9_CM_Diagnosis_Code, Fifth_ICD9_CM_Diagnosis_Code, PRC_DESC, primary_condition,
      CASE 
        WHEN pt.p_id is NOT NULL THEN 'PT'
        WHEN csec.p_id is NOT NULL THEN 'CS'
        WHEN h_cost.p_id is NOT NULL THEN 'HC'
        ELSE 'Other' END as label,
      CAST(AVG(Total_Charge_Per_Line) as Decimal(10,2)) as Total_Charge_Per_Line,
      h_cost.total_cost as total_cost
  FROM demo_user.WC_FR_V_NPATH_PRG_QRY a
  INNER JOIN(
    SELECT DISTINCT patient_identifier as p_id
    FROM demo_user.WC_FR_V_NPATH_PRG_QRY
    WHERE
        First_ICD9_CM_Diagnosis_Code like any ('64_._1','64_._2','65_._1','65_._2','66_._1','66_._2','67_._1','67_._2'
                                       'O10._2','O10._3','O11.3','O11.4','O12._4','O12._5','O13.4','O13.5','O14._4',
                                       'O14._5','O15.2','O16.4','O16.5','O24._2%','O24._3%','O25.2','O25.3','O26.62',
                                       'O26.63','O26.72','O26.73','O60.1%','O60.2%','O63.2','O70%','O80%','O82%',
                                       'O86.1%','O86.2%','O86.4%','O87%','O88._2','O88._3','O89%','O90%','O91._2',
                                       'O91._3','O92._2','O92._3','O98._2','O98._3','O9A._2','O9A._3','Z37%','Z39.0','Z39.1')
        OR Second_ICD9_CM_Diagnosis_Code like any ('64_._1','64_._2','65_._1','65_._2','66_._1','66_._2','67_._1','67_._2'
                                       'O10._2','O10._3','O11.3','O11.4','O12._4','O12._5','O13.4','O13.5','O14._4',
                                       'O14._5','O15.2','O16.4','O16.5','O24._2%','O24._3%','O25.2','O25.3','O26.62',
                                       'O26.63','O26.72','O26.73','O60.1%','O60.2%','O63.2','O70%','O80%','O82%',
                                       'O86.1%','O86.2%','O86.4%','O87%','O88._2','O88._3','O89%','O90%','O91._2',
                                       'O91._3','O92._2','O92._3','O98._2','O98._3','O9A._2','O9A._3','Z37%','Z39.0','Z39.1')
        OR Third_ICD9_CM_Diagnosis_Code like any ('64_._1','64_._2','65_._1','65_._2','66_._1','66_._2','67_._1','67_._2'
                                       'O10._2','O10._3','O11.3','O11.4','O12._4','O12._5','O13.4','O13.5','O14._4',
                                       'O14._5','O15.2','O16.4','O16.5','O24._2%','O24._3%','O25.2','O25.3','O26.62',
                                       'O26.63','O26.72','O26.73','O60.1%','O60.2%','O63.2','O70%','O80%','O82%',
                                       'O86.1%','O86.2%','O86.4%','O87%','O88._2','O88._3','O89%','O90%','O91._2',
                                       'O91._3','O92._2','O92._3','O98._2','O98._3','O9A._2','O9A._3','Z37%','Z39.0','Z39.1')
        OR Fourth_ICD9_CM_Diagnosis_Code like any ('64_._1','64_._2','65_._1','65_._2','66_._1','66_._2','67_._1','67_._2'
                                       'O10._2','O10._3','O11.3','O11.4','O12._4','O12._5','O13.4','O13.5','O14._4',
                                       'O14._5','O15.2','O16.4','O16.5','O24._2%','O24._3%','O25.2','O25.3','O26.62',
                                       'O26.63','O26.72','O26.73','O60.1%','O60.2%','O63.2','O70%','O80%','O82%',
                                       'O86.1%','O86.2%','O86.4%','O87%','O88._2','O88._3','O89%','O90%','O91._2',
                                       'O91._3','O92._2','O92._3','O98._2','O98._3','O9A._2','O9A._3','Z37%','Z39.0','Z39.1')
        OR Fifth_ICD9_CM_Diagnosis_Code like any ('64_._1','64_._2','65_._1','65_._2','66_._1','66_._2','67_._1','67_._2'
                                       'O10._2','O10._3','O11.3','O11.4','O12._4','O12._5','O13.4','O13.5','O14._4',
                                       'O14._5','O15.2','O16.4','O16.5','O24._2%','O24._3%','O25.2','O25.3','O26.62',
                                       'O26.63','O26.72','O26.73','O60.1%','O60.2%','O63.2','O70%','O80%','O82%',
                                       'O86.1%','O86.2%','O86.4%','O87%','O88._2','O88._3','O89%','O90%','O91._2',
                                       'O91._3','O92._2','O92._3','O98._2','O98._3','O9A._2','O9A._3','Z37%','Z39.0','Z39.1')
  ) d
  ON a.patient_identifier = d.p_id
  LEFT JOIN (
    SELECT DISTINCT patient_identifier as p_id
    FROM demo_user.WC_FR_V_NPATH_PRG_QRY
    WHERE
        First_ICD9_CM_Diagnosis_Code like any ('644%','O60%')
        OR Second_ICD9_CM_Diagnosis_Code like any ('644%','O60%')
        OR Third_ICD9_CM_Diagnosis_Code like any ('644%','O60%')
        OR Fourth_ICD9_CM_Diagnosis_Code like any ('644%','O60%')
        OR Fifth_ICD9_CM_Diagnosis_Code like any ('644%','O60%')
  ) pt
  ON a.patient_identifier = pt.p_id
  LEFT JOIN (
    SELECT DISTINCT patient_identifier as p_id
    FROM demo_user.WC_FR_V_NPATH_PRG_QRY
    WHERE
            (First_ICD9_CM_Diagnosis_Code in ('O82', 'Z38.01', 'Z38,62', 'Z38.64', 'Z38.66', 'Z38.31', 'Z38.69','740') OR 
            HCPCS_Line_Procedure_Billed_Code in ('59510','59514','59515')) OR 
            ICD9_CM_Principal_Procedure_Code in ('740','741','742','744','749','7499')
  ) csec
  ON a.patient_identifier = csec.p_id
  LEFT JOIN (
    SELECT patient_identifier as p_id,total_cost
    FROM (
        SELECT (Claim_Administrator_Claim_Number || '_' || Insurer_FEIN) as patient_identifier, 
        SUM(Total_Charge_Per_Line) total_cost
        FROM DEMO_HealthcareFWA.WC_MAT_PATIENT_ALL_BILLS 
        GROUP BY patient_identifier
    ) d
  ) h_cost
  ON a.patient_identifier = h_cost.p_id
  GROUP BY
      a.patient_identifier, HCPCS_Line_Procedure_Billed_Code, first_mat_diag, Service_Line_From_Date,
      Billing_Provider_Last_Name_or_Group, Billing_Provider_FEIN, First_ICD9_CM_Diagnosis_Code,
      Second_ICD9_CM_Diagnosis_Code, Third_ICD9_CM_Diagnosis_Code, Fourth_ICD9_CM_Diagnosis_Code,
      Fifth_ICD9_CM_Diagnosis_Code, PRC_DESC, primary_condition, label,total_cost
)
PARTITION BY patient_identifier
ORDER BY Service_Line_From_Date
USING
Mode (nonoverlapping)
Pattern ('(P|S|T|F|FF).(B){1,50}') 
Symbols (
  TRUE as B,
  CAST(First_ICD9_CM_Diagnosis_Code AS CHAR(3)) in ('630','631','632','633','634','635','636','637','638','639',
            '640','641','642','643','644','645','646','647','648','649',
            '650','651','652','653','654','655','656','657','658','659',
            '660','661','662','663','664','665','666','667','668','669',
            '670','671','672','673','674','675','676','677','678','679',
            'O00','O01','O02','O03','O04','O05','O06','O07','O08','O09',
            'O10','O11','O12','O13','O14','O15','O16','O17','O18','O19',
            'O20','O21','O22','O23','O24','O25','O26','O27','O28','O29',
            'O30','O31','O32','O33','O34','O35','O36','O37','O38','O39',
            'O40','O41','O42','O43','O44','O45','O46','O47','O48','O49',
            'O50','O51','O52','O53','O54','O55','O56','O57','O58','O59',
            'O60','O61','O62','O63','O64','O65','O66','O67','O68','O69',
            'O70','O71','O72','O73','O74','O75','O76','O77','O78','O79',
            'O80','O81','O82','O83','O84','O85','O86','O87','O88','O89',
            'O90','O91','O92','O93','O94','O95','O96','O97','O98','O99') as P,
  CAST(Second_ICD9_CM_Diagnosis_Code AS CHAR(3)) in ('630','631','632','633','634','635','636','637','638','639',
            '640','641','642','643','644','645','646','647','648','649',
            '650','651','652','653','654','655','656','657','658','659',
            '660','661','662','663','664','665','666','667','668','669',
            '670','671','672','673','674','675','676','677','678','679',
            'O00','O01','O02','O03','O04','O05','O06','O07','O08','O09',
            'O10','O11','O12','O13','O14','O15','O16','O17','O18','O19',
            'O20','O21','O22','O23','O24','O25','O26','O27','O28','O29',
            'O30','O31','O32','O33','O34','O35','O36','O37','O38','O39',
            'O40','O41','O42','O43','O44','O45','O46','O47','O48','O49',
            'O50','O51','O52','O53','O54','O55','O56','O57','O58','O59',
            'O60','O61','O62','O63','O64','O65','O66','O67','O68','O69',
            'O70','O71','O72','O73','O74','O75','O76','O77','O78','O79',
            'O80','O81','O82','O83','O84','O85','O86','O87','O88','O89',
            'O90','O91','O92','O93','O94','O95','O96','O97','O98','O99') as S,
  CAST(Third_ICD9_CM_Diagnosis_Code AS CHAR(3)) in ('630','631','632','633','634','635','636','637','638','639',
            '640','641','642','643','644','645','646','647','648','649',
            '650','651','652','653','654','655','656','657','658','659',
            '660','661','662','663','664','665','666','667','668','669',
            '670','671','672','673','674','675','676','677','678','679',
            'O00','O01','O02','O03','O04','O05','O06','O07','O08','O09',
            'O10','O11','O12','O13','O14','O15','O16','O17','O18','O19',
            'O20','O21','O22','O23','O24','O25','O26','O27','O28','O29',
            'O30','O31','O32','O33','O34','O35','O36','O37','O38','O39',
            'O40','O41','O42','O43','O44','O45','O46','O47','O48','O49',
            'O50','O51','O52','O53','O54','O55','O56','O57','O58','O59',
            'O60','O61','O62','O63','O64','O65','O66','O67','O68','O69',
            'O70','O71','O72','O73','O74','O75','O76','O77','O78','O79',
            'O80','O81','O82','O83','O84','O85','O86','O87','O88','O89',
            'O90','O91','O92','O93','O94','O95','O96','O97','O98','O99') as T,
  CAST(Fourth_ICD9_CM_Diagnosis_Code AS CHAR(3)) in ('630','631','632','633','634','635','636','637','638','639',
            '640','641','642','643','644','645','646','647','648','649',
            '650','651','652','653','654','655','656','657','658','659',
            '660','661','662','663','664','665','666','667','668','669',
            '670','671','672','673','674','675','676','677','678','679',
            'O00','O01','O02','O03','O04','O05','O06','O07','O08','O09',
            'O10','O11','O12','O13','O14','O15','O16','O17','O18','O19',
            'O20','O21','O22','O23','O24','O25','O26','O27','O28','O29',
            'O30','O31','O32','O33','O34','O35','O36','O37','O38','O39',
            'O40','O41','O42','O43','O44','O45','O46','O47','O48','O49',
            'O50','O51','O52','O53','O54','O55','O56','O57','O58','O59',
            'O60','O61','O62','O63','O64','O65','O66','O67','O68','O69',
            'O70','O71','O72','O73','O74','O75','O76','O77','O78','O79',
            'O80','O81','O82','O83','O84','O85','O86','O87','O88','O89',
            'O90','O91','O92','O93','O94','O95','O96','O97','O98','O99') as F,
  CAST(Fifth_ICD9_CM_Diagnosis_Code AS CHAR(3)) in ('630','631','632','633','634','635','636','637','638','639',
            '640','641','642','643','644','645','646','647','648','649',
            '650','651','652','653','654','655','656','657','658','659',
            '660','661','662','663','664','665','666','667','668','669',
            '670','671','672','673','674','675','676','677','678','679',
            'O00','O01','O02','O03','O04','O05','O06','O07','O08','O09',
            'O10','O11','O12','O13','O14','O15','O16','O17','O18','O19',
            'O20','O21','O22','O23','O24','O25','O26','O27','O28','O29',
            'O30','O31','O32','O33','O34','O35','O36','O37','O38','O39',
            'O40','O41','O42','O43','O44','O45','O46','O47','O48','O49',
            'O50','O51','O52','O53','O54','O55','O56','O57','O58','O59',
            'O60','O61','O62','O63','O64','O65','O66','O67','O68','O69',
            'O70','O71','O72','O73','O74','O75','O76','O77','O78','O79',
            'O80','O81','O82','O83','O84','O85','O86','O87','O88','O89',
            'O90','O91','O92','O93','O94','O95','O96','O97','O98','O99') as FF
)
Result (

  FIRST (patient_identifier OF ANY(P,S,T,F,FF,B)) AS patient_identifier,
  ACCUMULATE (CAST (PRC_DESC AS VARCHAR(40)) OF ANY(P,S,T,F,FF,B)) AS path,
  ACCUMULATE(Total_Charge_Per_Line of ANY(P,S,T,F,FF,B)) as cost,
  FIRST(total_cost of any(P,S,T,F,FF,B)) as total_cost,   
  FIRST(label of any(P,S,T,F,FF,B)) as label,
  ACCUMULATE(Billing_Provider_Last_Name_or_Group of ANY(P,S,T,F,FF,B)) as prov_group,
  ACCUMULATE(Billing_Provider_FEIN of ANY(P,S,T,F,FF,B)) as provider
)
) p
) WITH DATA ;  

<p style = 'font-size:16px;font-family:Arial'> Below table calculates the chapterwise cost for maternity procedures.</p>

In [ ]:
CREATE TABLE DEMO_USER.mat_chap_comp  as (
SELECT 
chap, SUM(total_cost) AS total_cost, SUM(patients) AS patients, (SUM(avg_cost)/2) AS avg_cost 
FROM
(
SELECT CASE WHEN chapter like '64%' THEN 'Normal Delivery' ELSE 'Delivery with Complications' END as chap,
SUM(Total_Charge_Per_Line) total_cost, COUNT(DISTINCT patient_identifier) patients, (total_cost/patients) avg_cost
FROM (
    SELECT *
    FROM(
        select
            (Claim_Administrator_Claim_Number || '_' || Insurer_FEIN) as patient_identifier,
            CAST(First_ICD9_CM_Diagnosis_Code AS CHAR(3)) as chp_1, CAST(Second_ICD9_CM_Diagnosis_Code AS CHAR(3)) as chp_2,
            CAST(Third_ICD9_CM_Diagnosis_Code AS CHAR(3)) as chp_3, CAST(Fourth_ICD9_CM_Diagnosis_Code AS CHAR(3)) as chp_4,
            CAST(Fifth_ICD9_CM_Diagnosis_Code AS CHAR(3)) as chp_5, Total_Charge_Per_Line
        from DEMO_HealthcareFWA.WC_SV1_MAT
        UNION ALL
        select
            (Claim_Administrator_Claim_Number || '_' || Insurer_FEIN) as patient_identifier,
            CAST(First_ICD9_CM_Diagnosis_Code AS CHAR(3)) as chp_1, CAST(Second_ICD9_CM_Diagnosis_Code AS CHAR(3)) as chp_2,
            CAST(Third_ICD9_CM_Diagnosis_Code AS CHAR(3)) as chp_3, CAST(Fourth_ICD_9_CM_Diagnosis_Code AS CHAR(3)) as chp_4,
            CAST(Fifth_ICD9_CM_Diagnosis_Code AS CHAR(3)) as chp_5, CAST(Total_Charge_Per_Line AS FLOAT)
        from DEMO_HealthcareFWA.WC_SV2_MAT
    ) d
    UNPIVOT EXCLUDE NULLS
    (
        chapter FOR diag_level IN
        (chp_1, chp_2, chp_3, chp_4, chp_5)
    ) Tmp
    where CAST(chapter AS CHAR(2)) in ('64','65')--,'67','O0','O1','O2','O3','O4','O5','O6','O7','O8','O9','V2','Z3','76','77')
) up
GROUP BY chap

UNION

SELECT CASE WHEN chapter in ('Z34','O80','O82') THEN 'Normal Delivery' ELSE 'Delivery with Complications' END as chap,
SUM(Total_Charge_Per_Line) total_cost, COUNT(DISTINCT patient_identifier) patients, (total_cost/patients) avg_cost
FROM (
    SELECT *
    FROM(
        select
            (Claim_Administrator_Claim_Number || '_' || Insurer_FEIN) as patient_identifier,
            CAST(First_ICD9_CM_Diagnosis_Code AS CHAR(3)) as chp_1, CAST(Second_ICD9_CM_Diagnosis_Code AS CHAR(3)) as chp_2,
            CAST(Third_ICD9_CM_Diagnosis_Code AS CHAR(3)) as chp_3, CAST(Fourth_ICD9_CM_Diagnosis_Code AS CHAR(3)) as chp_4,
            CAST(Fifth_ICD9_CM_Diagnosis_Code AS CHAR(3)) as chp_5, Total_Charge_Per_Line
        from DEMO_HealthcareFWA.WC_SV1_MAT
        UNION ALL
        select
            (Claim_Administrator_Claim_Number || '_' || Insurer_FEIN) as patient_identifier,
            CAST(First_ICD9_CM_Diagnosis_Code AS CHAR(3)) as chp_1, CAST(Second_ICD9_CM_Diagnosis_Code AS CHAR(3)) as chp_2,
            CAST(Third_ICD9_CM_Diagnosis_Code AS CHAR(3)) as chp_3, CAST(Fourth_ICD_9_CM_Diagnosis_Code AS CHAR(3)) as chp_4,
            CAST(Fifth_ICD9_CM_Diagnosis_Code AS CHAR(3)) as chp_5, CAST(Total_Charge_Per_Line AS FLOAT)
        from DEMO_HealthcareFWA.WC_SV2_MAT
    ) d
    UNPIVOT EXCLUDE NULLS
    (
        chapter FOR diag_level IN
        (chp_1, chp_2, chp_3, chp_4, chp_5)
    ) Tmp
    where chapter in ('Z34','O80','O82','O60','O61','O62','O63','O64','O65','O66','O67','O68','O69','O70','O71','O72','O73','O74','O75','O76','O77')--,'67','O0','O1','O2','O3','O4','O5','O6','O7','O8','O9','V2','Z3','76','77')
) up
GROUP BY chap) CombineResults
GROUP BY chap
) WITH DATA ;                 

<p style = 'font-size:16px;font-family:Arial'>Below table lists the top maternity procedures.</p>

In [ ]:
CREATE TABLE DEMO_USER.mat_top_proc  as (
SELECT
    RANK() OVER (ORDER BY TOTAL_COST DESC) as COST_RANK,
    CAST(Description AS VARCHAR(45)) as "Procedure", TOTAL_COST
FROM (
    select 
        COALESCE("Sub_Category","SHORT DESCRIPTION") as Description, 
        CAST(SUM(Total_Charge_Per_Line) AS DECIMAL(20,2)) as TOTAL_COST
    FROM (
        select
            CAST(Total_Charge_Per_Line AS FLOAT) as Total_Charge_Per_Line,
            HCPCS_Line_Procedure_Billed_Code, CAST(Date_of_Bill as CHAR(4)) as DOB
        from DEMO_HealthcareFWA.WC_SV1_MAT
        WHERE HCPCS_Line_Procedure_Billed_Code is not NULL

        UNION ALL

        select
            CAST(Total_Charge_Per_Line AS FLOAT) as Total_Charge_Per_Line,
            HCPCS_Line_Procedure_Billed_Code, CAST(Date_of_Bill as CHAR(4)) as DOB
        from DEMO_HealthcareFWA.WC_SV2_MAT
        WHERE HCPCS_Line_Procedure_Billed_Code is not NULL
    ) d
    LEFT JOIN DEMO_HealthcareFWA.WC_CPT_Coding_temp
    ON
        (HCPCS_Line_Procedure_Billed_Code BETWEEN "FROM" AND COALESCE("TO","FROM")) AND
            (LENGTH(HCPCS_Line_Procedure_Billed_Code) = LENGTH("FROM"))
    LEFT JOIN DEMO_HealthcareFWA.WC_HCPCS_Coding
    ON HCPCS_Line_Procedure_Billed_Code = HCPC
    GROUP BY Description
) d
QUALIFY COST_RANK <= 10
) WITH DATA ;                  

<p style = 'font-size:16px;font-family:Arial'>Below tables lists the maternity provider information.</p>

In [ ]:
CREATE TABLE DEMO_USER.mat_path_prov  as (
SELECT
Billing_Provider_FEIN,
SUM(CASE 
  WHEN First_ICD9_CM_Diagnosis_Code in ('O82', 'Z38.01', 'Z38,62', 'Z38.64', 'Z38.66', 'Z38.31', 'Z38.69','740') THEN 1
  WHEN Second_ICD9_CM_Diagnosis_Code in ('O82', 'Z38.01', 'Z38,62', 'Z38.64', 'Z38.66', 'Z38.31', 'Z38.69','740') THEN 1
  WHEN Third_ICD9_CM_Diagnosis_Code in ('O82', 'Z38.01', 'Z38,62', 'Z38.64', 'Z38.66', 'Z38.31', 'Z38.69','740') THEN 1
  WHEN Fourth_ICD9_CM_Diagnosis_Code in ('O82', 'Z38.01', 'Z38,62', 'Z38.64', 'Z38.66', 'Z38.31', 'Z38.69','740') THEN 1
  WHEN Fifth_ICD9_CM_Diagnosis_Code in ('O82', 'Z38.01', 'Z38,62', 'Z38.64', 'Z38.66', 'Z38.31', 'Z38.69','740') THEN 1
  WHEN HCPCS_Line_Procedure_Billed_Code in ('59510','59514','59515') THEN 1
  WHEN ICD9_CM_Principal_Procedure_Code in ('740','741','742','744','749','7499') THEN 1
  ELSE 0
  END
) AS "Total Cesarean Deliveries",
SUM(CASE WHEN HCPCS_Line_Procedure_Billed_Code BETWEEN '76801' AND '76817' AND LENGTH(HCPCS_Line_Procedure_Billed_Code) = 5 THEN 1 ELSE 0 END) "Total Ultrasound Performed",
SUM(CASE WHEN HCPCS_Line_Procedure_Billed_Code in ('99291','99292') then 1 ELSE 0 END) "Total Neonatal ICU Admissions",
SUM(CASE WHEN HCPCS_Line_Procedure_Billed_Code in ('82105') then 1 ELSE 0 END) as "Total Alpha Fetoprotein Tests",
SUM(Total_Charge_Per_Line) "Total Costs"
FROM (
select
  Billing_Provider_FEIN, Total_Charge_Per_Line,
  First_ICD9_CM_Diagnosis_Code,Second_ICD9_CM_Diagnosis_Code,Third_ICD9_CM_Diagnosis_Code,Fourth_ICD9_CM_Diagnosis_Code,
  Fifth_ICD9_CM_Diagnosis_Code,ICD9_CM_Principal_Procedure_Code,HCPCS_Line_Procedure_Billed_Code
from DEMO_HealthcareFWA.WC_MAT_PATIENT_ALL_BILLS
) d
GROUP BY Billing_Provider_FEIN
 ) WITH DATA ;    
    

<p style = 'font-size:16px;font-family:Arial'> Below table gives information on maternity facility.</p>

In [ ]:
CREATE TABLE DEMO_USER.mat_top_facil  as (
SELECT 
    RANK() OVER (ORDER BY COST DESC) as COST_RANK, d.Billing_Provider_FEIN,
    Facility_Name, Billing_Provider_First_Name, Billing_Provider_Last_Name_or_Group, COST
FROM
(
    SELECT Billing_Provider_FEIN, CAST(SUM(Total_Charge_Per_Line) AS DECIMAL(20,2)) as COST
    FROM (
        select 
            CAST(Total_Charge_Per_Line AS FLOAT) as Total_Charge_Per_Line, Billing_Provider_FEIN,
            CAST(Date_of_Bill as CHAR(4)) as DOB
        from DEMO_HealthcareFWA.WC_SV1_MAT

        UNION ALL

        select
            CAST(Total_Charge_Per_Line AS FLOAT) as Total_Charge_Per_Line, Billing_Provider_FEIN,
            CAST(Date_of_Bill as CHAR(4)) as DOB
        from DEMO_HealthcareFWA.WC_SV2_MAT
    ) d
    GROUP BY Billing_Provider_FEIN
) d
LEFT JOIN DEMO_HealthcareFWA.WC_PROVIDER_LOOKUP lu
ON lu.Billing_Provider_FEIN = d.Billing_Provider_FEIN
QUALIFY COST_RANK <= 10
) WITH DATA ;                 ;

<p style = 'font-size:18px;font-family:Arial'><b>Conclusion:</b></p>
<p style = 'font-size:16px;font-family:Arial'>
In this notebook we have created the tables which are then used in the dashboard for the reporting. Here we can see how we have used Teradata's inbuild functions to calculate and derive complex logic and calutions. The dataprep step makes it seamless and easy to grasp the insights using visual dashboards.<br> Please review the insights derived from the source data and assess how they might assist in identifying Fraud, Waste, and Abuse (FWA).</p>
<p style = 'font-size:16px;font-family:Arial'>
<div style="text-align: left; ">
        <a href="./FWA_Dataprocessing_sql.ipynb" style="padding: 10px; background-color: #00233c; color: #FFFFFF; text-decoration: none; border-radius: 5px;">
         Superset Dashboards
        </a> </div></p>


<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial'>7. Cleanup</b>

<p style = 'font-size:18px;font-family:Arial'><b>Work Tables</b></p>
<p style = 'font-size:16px;font-family:Arial'>Cleanup work tables to prevent errors next time. This section drops all the views and tables created during the demonstration.</p>

In [ ]:
drop view demo_user.wc_bk_v_npath_qry;

In [ ]:
drop view demo_user.va_bk_v_pat_enc_lt_90d;

In [ ]:
drop view demo_user.va_bk_v_a1c_enc;

In [ ]:
drop view demo_user.va_bk_v_diabetes_codes;

In [ ]:
drop view demo_user.va_bk_diab_strt;

In [ ]:
drop view demo_user.va_bk_v_a1c_per_yr;

In [ ]:
drop view demo_user.va_fr_v_enc_costs;

In [ ]:
drop view demo_user.va_bk_v_sel_hrt_rnl;

In [ ]:
drop view demo_user.wc_fr_v_npath_prg_qry;

In [ ]:
drop table demo_user.chapter_summary ;

In [ ]:
drop table demo_user.overview_chapters ;

In [ ]:
drop table demo_user.overview_procedures ;

In [ ]:
drop table demo_user.overview_providers ;

In [ ]:
drop table demo_user.msk_comorb_conds ;

In [ ]:
drop table demo_user.msk_proc_groups ;

In [ ]:
drop table demo_user.msk_procedures ;

In [ ]:
drop table demo_user.msk_providers ;

In [ ]:
drop table demo_user.msk_drilldown_proc_groups ;

In [ ]:
drop table demo_user.msk_drilldown_proc_grp_counties ;

In [ ]:
drop table demo_user.msk_drilldown_proc_grp_procedures ;

In [ ]:
drop table demo_user.msk_drilldown_proc_grp_provs ;

In [ ]:
drop table demo_user.srg_averages ;

In [ ]:
drop table demo_user.srg_mri ;

In [ ]:
drop table demo_user.msk_pathwys_prov ;

In [ ]:
drop table demo_user.surg_pathways ;

In [ ]:
drop table demo_user.nsurg_pathways ;

In [ ]:
drop table demo_user.Total_cost_for_procedure_groups_dbt ;

In [ ]:
drop table demo_user.Total_cost_for_procedures_dbt ;

In [ ]:
drop table demo_user.Hemoglobin_count_dbt ;

In [ ]:
drop table demo_user.Total_cost_for_facilities_dbt ;

In [ ]:
drop table demo_user.dbt_drilldown_proc_groups ;

In [ ]:
drop table demo_user.dbt_drilldown_proc_grp_procedures ;

In [ ]:
drop table demo_user.dbt_drilldown_proc_grp_counties ;

In [ ]:
drop table demo_user.dbt_drilldown_proc_grp_provs ;

In [ ]:
drop table demo_user.dbt_pathways ;

In [ ]:
drop table demo_user.dbt_pathwys_prov ;

In [ ]:
drop table demo_user.mat_chap_comp ;

In [ ]:
drop table demo_user.mat_proc_grps ;

In [ ]:
drop table demo_user.mat_top_proc ;

In [ ]:
drop table demo_user.mat_top_facil ;

In [ ]:
drop table demo_user.mat_pathways ;

In [ ]:
drop table demo_user.mat_path_prov ;

<hr style="height:1px;border:none;">
<p style = 'font-size:18px;font-family:Arial'> <b>Databases and Tables </b></p>
<p style = 'font-size:16px;font-family:Arial'>The following code will clean up tables and databases created above.</p>

In [ ]:
call remove_data('DEMO_HealthcareFWA');      

<footer style="padding-bottom:35px; border-bottom:3px solid #91A0Ab">
    <div style="float:left;margin-top:14px">ClearScape Analytics™</div>
    <div style="float:right;">
        <div style="float:left; margin-top:14px">
            Copyright © Teradata Corporation - 2025. All Rights Reserved
        </div>
    </div>
</footer>